In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm, SymLogNorm, TwoSlopeNorm
import matplotlib.colors as colors
plt.rcParams['figure.max_open_warning'] = 500
import qcodes as qc
import qcodes.instrument_drivers.nplab_drivers as npd
from scipy.optimize import curve_fit
from scipy.interpolate import interp2d, interp1d
from os import path
import colorcet as cc   # install quickly with "conda install colorcet"
from scipy.interpolate import RectBivariateSpline
from matplotlib import ticker
from scipy.ndimage import gaussian_filter, uniform_filter
import matplotlib as mpl
from lmfit import Model
import glob

In [2]:
plt.rcParams['svg.fonttype'] = 'none'
plt.rcParams['figure.dpi'] = 300
plt.rcParams['lines.markersize'] = 4
%config InlineBackend.figure_format = 'svg'

font = {
                'family': 'Arial',
                'size': 12,
                'serif': 'Times',
                'sans-serif': 'Arial'
            }

plt.rc('font', **font)

In [3]:
cd ..

/Users/robertpolski/Documents/NPGroupResearch/Measurements/Triton/qcodes_data


In [4]:
qc.DataSet.default_io.base_location = path.abspath('.')

In [5]:
figfile = 'Figures/M30_analysis/'

In [6]:
%matplotlib widget

Here is what `zeroDparam` looks like

    def zerofieldsw(Vb):
        slDrc = -0.2058088083627785
        srframe.volt_p1(Vb)
        srframe.volt_p5(-slDrc*Vb)

    zeroDparam = qc.Parameter('zeroDparam', set_cmd=zerofieldsw)
    
Then the other colored bars are offset from this by a constant

    def constant_D_redline_nsw(Vb):
        slDrc = -0.2058088083627785
        srframe.volt_p1(Vb)
        srframe.volt_p5(-slDrc*Vb - 2.2)

    constant_D_redline_nsw = qc.Parameter('constant_D_redline_nsw', set_cmd=constant_D_redline_nsw)

    def constant_D_greenline_nsw(Vb):
        slDrc = -0.2058088083627785
        srframe.volt_p1(Vb)
        srframe.volt_p5(-slDrc*Vb - 1.2)

    constant_D_greenline_nsw = qc.Parameter('constant_D_greenline_nsw', set_cmd=constant_D_greenline_nsw)

    def constant_D_greyline_nsw(Vb):
        slDrc = -0.2058088083627785
        srframe.volt_p1(Vb)
        srframe.volt_p5(-slDrc*Vb + 1.3)
    
    constant_D_greyline_nsw = qc.Parameter('constant_D_greyline_nsw', set_cmd=constant_D_greyline_nsw)

    def constant_D_orangeline_nsw(Vb):
        slDrc = -0.2058088083627785
        srframe.volt_p1(Vb)
        srframe.volt_p5(-slDrc*Vb + 2.9)
    
    constant_D_orangeline_nsw = qc.Parameter('constant_D_orangeline_nsw', set_cmd=constant_D_orangeline_nsw)

# M30 determination of charge neutrality, displacement field

Lockin865 was on electrodes 29-28 (superconductivity), lockin830 on 40-41 (integer-filling insulating states)

Both R$_{xx}$ measurements

In [7]:
d2d = qc.load_data('data/2020-05-05/#024_M30_865_29-28__830_40-41_0p5nA_2dgatesweep_19-52-29')
d2d

DataSet:
   location = 'data/2020-05-05/#024_M30_865_29-28__830_40-41_0p5nA_2dgatesweep_19-52-29'
   <Type>   | <array_id>          | <array.name> | <array.shape>
   Setpoint | srframe_volt_p5_set | volt_p5      | (193,)
   Setpoint | srframe_volt_p1_set | volt_p1      | (193, 272)
   Measured | lockin830_X         | X            | (193, 272)
   Measured | lockin830_Y         | Y            | (193, 272)
   Measured | lockin865_X         | X            | (193, 272)
   Measured | lockin865_Y         | Y            | (193, 272)

In [8]:
R2928_2d = npd.Rxxfromdata(d2d, 0.5e-9, 'lockin865')[:, 6:]
R4041_2d = npd.Rxxfromdata(d2d, 0.5e-9, 'lockin830')[:, 6:]

tg2d = d2d.srframe_volt_p5_set[:]
bg2d = d2d.srframe_volt_p1_set[0, 6:]

In [9]:
fig, ax = plt.subplots(1, 2, figsize=(10, 4))
dnorm = TwoSlopeNorm(vcenter=4, vmin=0, vmax=70)
ax[0].pcolormesh(bg2d, tg2d, R2928_2d/1000, cmap=cc.cm.diverging_bwr_20_95_c54, norm=dnorm)
ax[1].pcolormesh(bg2d, tg2d, R4041_2d/1000, cmap=cc.cm.diverging_bwr_20_95_c54, norm=dnorm)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
# the slope of the charge neutrality line
slCNP = -0.2058088083627785
x1 = np.array([-10, 2])
ax[0].plot(x1, x1*slCNP - 1.27, 'k--')
ax[1].plot(x1, x1*slCNP - 1.27, 'k--')

From here we get get the 0-displacement-field line. The equations of interest are:

\begin{eqnarray}
n & = & \left[C_t V_t + C_b V_b\right]/e + n_0\\
D & = & \left[C_t V_t - C_b V_b\right]/\epsilon_0
\end{eqnarray}

This assumes that when topgate and backgate are 0, there is no D field. So a $D = 0$ line will pass through the point (V_b, V_t) = (0, 0). With some algebra, we get an equation for the line:

\begin{equation}
V_t = \frac{C_b}{C_t} V_b = m_1 V_b
\end{equation}

Now we want to relate this slope $m_1$ to the slope of the $n = 0$ line, with the equation

\begin{equation}
V_t = -\frac{C_b}{C_t} V_b - \frac{e}{C_t} n_0 = m_2 V_b + b_2
\end{equation}

We see that $m_1 = -m_2$, so now we know the $D = 0$ line.

In [11]:
# plot the D = 0 lines
x2 = np.array([-7.5, 5])
ax[0].plot(x2, -slCNP*x2, '--', color='orange')
ax[1].plot(x2, -slCNP*x2, '--', color='orange')

This was plotted in fan diagrams, where the parameter zeroDparam sets the backgate to the setpoint $s$ and the topgate to $-m_2 s$, or $0.20581 s$

In [12]:
# first a single sweep
d0sweep = qc.load_data('data/2020-05-06/#002_M30_865_29-28__830_40-41_0p5nA_0T_D0sweep_20-54-51')
d0sweep

DataSet:
   location = 'data/2020-05-06/#002_M30_865_29-28__830_40-41_0p5nA_0T_D0sweep_20-54-51'
   <Type>   | <array_id>     | <array.name> | <array.shape>
   Setpoint | zeroDparam_set | zeroDparam   | (221,)
   Measured | lockin830_X    | X            | (221,)
   Measured | lockin830_Y    | Y            | (221,)
   Measured | lockin865_X    | X            | (221,)
   Measured | lockin865_Y    | Y            | (221,)

In [13]:
plt.figure(figsize=(8, 4))
plt.semilogy(d0sweep.zeroDparam_set, npd.Rxxfromdata(d0sweep, 0.5e-9, 'lockin865'), label='29-28')
plt.semilogy(d0sweep.zeroDparam_set, npd.Rxxfromdata(d0sweep, 0.5e-9, 'lockin830'), label='40-41')
plt.ylim(20, 5e5)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(20, 500000.0)

Now the fan diagrams to determine the angle. The capacitance of the gates (and therefore n and D) can be determined by inserting the $D = 0$ equation above back into $n$:

\begin{equation}
n = \frac{2C_b}{e} V_b + n_0
\end{equation}

So from the fan diagrams, we can determine $2C_b$, and everything else follows from there

In [14]:
dfand0 = qc.load_data('data/2020-05-06/#005_M30_865_29-28__830_40-41_1nA_baseT_B-n_D0_sweep_23-13-35')
Rxx2928d0 = npd.Rxxfromdata(dfand0, 0.5e-9, 'lockin865')
Rxx4041d0 = npd.Rxxfromdata(dfand0, 0.5e-9, 'lockin830')
Vb = dfand0.zeroDparam_set[0, :]
B = np.array(dfand0.triton_field_set_stable_set)
dfand0

DataSet:
   location = 'data/2020-05-06/#005_M30_865_29-28__830_40-41_1nA_baseT_B-n_D0_sweep_23-13-35'
   <Type>   | <array_id>                  | <array.name>     | <array.shape>
   Setpoint | triton_field_set_stable_set | field_set_stable | (161,)
   Setpoint | zeroDparam_set              | zeroDparam       | (161, 451)
   Measured | lockin830_X                 | X                | (161, 451)
   Measured | lockin830_Y                 | Y                | (161, 451)
   Measured | lockin865_X                 | X                | (161, 451)
   Measured | lockin865_Y                 | Y                | (161, 451)

## Electrodes 29-28

In [15]:
fig, ax = plt.subplots(1, 1, figsize=(10, 5))
normf = TwoSlopeNorm(vcenter=10, vmin=0, vmax=70)
ax.pcolormesh(Vb, B, Rxx2928d0/1000, cmap=cc.cm.diverging_bwr_20_95_c54, norm=normf)
# ax[1].pcolormesh(Vb, B, Rxx4041d0/1000, cmap=cc.cm.diverging_bwr_20_95_c54, norm=normf)


Bl = 7
Bs = 0
V0 = -3.002
sl = 30
b = -V0*sl

for i in [-12, -8, -4, -3, -2, -1]:
    x = np.array([(Bs*i - b)/sl, (Bl*i - b)/sl])
    ax.plot(x, (x*sl + b)/i, color='white', lw=0.5)

V0p = -2.99
b = -V0p*sl
for i in [1, 2, 3, 4, 6, 8, 10]:
    x = np.array([(Bs*i - b)/sl, (Bl*i - b)/sl])
    ax.plot(x, (x*sl + b)/i, color='white', lw=0.5)


Vfn = -5.93
b = -Vfn*sl
for i in [-4, -8]:
    x = np.array([(Bs*i - b)/sl, (Bl*i - b)/sl])
    ax.plot(x, (x*sl + b)/i, color='white', lw=0.5)
    
Vfp = 0
b = -Vfp*sl
for i in [4, 8]:
    x = np.array([(Bs*i - b)/sl, (Bl*i - b)/sl])
    ax.plot(x, (x*sl + b)/i, color='white', lw=0.5)  

# plt.axis(a)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
a = plt.axis()

Then we can get the capacitance using 

$$ B = slope (n - n_0) = \frac{h}{e}(n - n_0) = \frac{2hC_b}{e^2}V_b$$

In [17]:
# Then we can get the capacitance
V0 = -3
sl = 30
Cb = sl*1.602e-19**2/6.626e-34/2
Ct = Cb/(-slCNP)
dBNb = 3*8.85e-12/Cb
dBNt = 3*8.85e-12/Ct
print('Bottom capacitance (C/(V*m^2)): ', str(Cb))
print('Bottom effective hBN thickness (nm): ', str(dBNb*10**9))
print('Top capacitance (C/(V*m^2)): ', str(Ct))
print('Top effective hBN thickness (nm): ', str(dBNt*10**9))

Bottom capacitance (C/(V*m^2)):  0.0005809849079384244
Bottom effective hBN thickness (nm):  45.69826106879509
Top capacitance (C/(V*m^2)):  0.0028229350947619513
Top effective hBN thickness (nm):  9.405104654819871


In [18]:
# We can replot in density now
Cb = 0.0005809849079384244
prefactor = 2*Cb/1.602e-19/10**16
nd0_2829 = prefactor*(Vb - V0) # in 10^12 cm^-2

In [19]:
fig, ax = plt.subplots(1, 1, figsize=(10, 5))
normf = TwoSlopeNorm(vcenter=3, vmin=0, vmax=20)
ax.pcolormesh(nd0_2829, B, Rxx2928d0/1000, cmap=cc.cm.diverging_bwr_20_95_c54, norm=normf)
# ax[1].pcolormesh(Vb, B, Rxx4041d0/1000, cmap=cc.cm.diverging_bwr_20_95_c54, norm=normf)


Bl = 7
Bs = 0
n0 = 0
sl = 30/prefactor
b = -n0*sl

for i in [-12, -8, -4, -3, -2, -1]:
    x = np.array([(Bs*i - b)/sl, (Bl*i - b)/sl])
    ax.plot(x, (x*sl + b)/i, color='white', lw=0.5)

n0p = 0
b = -n0p*sl
for i in [1, 2, 3, 4, 6, 8, 10]:
    x = np.array([(Bs*i - b)/sl, (Bl*i - b)/sl])
    ax.plot(x, (x*sl + b)/i, color='white', lw=0.5)


nfn = -2.13
b = -nfn*sl
for i in [-4, -8]:
    x = np.array([(Bs*i - b)/sl, (Bl*i - b)/sl])
    ax.plot(x, (x*sl + b)/i, color='white', lw=0.5)
    
# nfp = 2.17
# b = -nfp*sl
# for i in [4, 8]:
#     x = np.array([(Bs*i - b)/sl, (Bl*i - b)/sl])
#     ax.plot(x, (x*sl + b)/i, color='white', lw=0.5)  

# plt.axis(a)

# and from half filling, we get nhfp = 1.02, so nfp = between 2.04 and 2.13

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:
nfull = (2.04 + 2.13)/2*10**12 # in cm^-2
a = 0.246e-9
theta = np.sqrt(np.sqrt(3)*nfull*10**4*a**2/8)*180/np.pi
print('theta: ', theta)

nfullbound0 = 2.04*10**12 # in cm^-2
a = 0.246e-9
thetabound0 = np.sqrt(np.sqrt(3)*nfullbound0*10**4*a**2/8)*180/np.pi
print('theta bottom bound, based on half filling insulator: ', thetabound0)

nfullbound1 = 2.13*10**12 # in cm^-2
a = 0.246e-9
thetabound1 = np.sqrt(np.sqrt(3)*nfullbound1*10**4*a**2/8)*180/np.pi
print('theta top bound, based on full filling fan: ', thetabound1)

theta:  0.9469915184650085
theta bottom bound, based on half filling insulator:  0.9367164420574015
theta top bound, based on full filling fan:  0.9571562982442566


## Electrodes 40-41

In [21]:
V04041 = -3.04
prefactor = 2*Cb/1.602e-19/10**16
nd0_4041 = prefactor*(Vb - V04041) # in 10^12 cm^-2

In [22]:
fig, ax = plt.subplots(1, 1, figsize=(10, 5))
normf = TwoSlopeNorm(vcenter=40, vmin=-1, vmax=60)
ax.pcolormesh(nd0_4041, B, Rxx4041d0/1000, cmap=cc.cm.diverging_bwr_20_95_c54, norm=normf)


Bl = 7
Bs = 0
n0 = 0
sl = 30/prefactor
b = -n0*sl

for i in [-10, -8, -4, -3, -2, -1]:
    x = np.array([(Bs*i - b)/sl, (Bl*i - b)/sl])
    ax.plot(x, (x*sl + b)/i, color='white', lw=0.5)

n0p = 0
b = -n0p*sl
for i in [1, 2, 3, 4, 6]:
    x = np.array([(Bs*i - b)/sl, (Bl*i - b)/sl])
    ax.plot(x, (x*sl + b)/i, color='white', lw=0.5)

nfp = 2.15
b = -nfp*sl
for i in [4, 6, 8, 12]:
    x = np.array([(Bs*i - b)/sl, (Bl*i - b)/sl])
    ax.plot(x, (x*sl + b)/i, color='white', lw=0.5)

nhfn = -1.21
b = -nhfn*sl
for i in [-2, -4, -6, -8, -10]:
    x = np.array([(Bs*i - b)/sl, (Bl*i - b)/sl])
    ax.plot(x, (x*sl + b)/i, color='white', lw=0.5)  


# nhfn = -1.
# b = -nhfn*sl
# for i in [-2, -4, -6, -8, -10]:
#     x = np.array([(Bs*i - b)/sl, (Bl*i - b)/sl])
#     ax.plot(x, (x*sl + b)/i, color='white', lw=0.5)  

# plt.axis(a)

# and from half filling, we get nhfp = -1.0 or -1.22. so nfp around 2.0 - 2.44 (positive full filling resembles ~2.3, perhaps a good midpoint)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [23]:
# a = plt.axis()

In [24]:
nfull2 = 2.3*10**12 # in cm^-2
a = 0.246e-9
theta2 = np.sqrt(np.sqrt(3)*nfull2*10**4*a**2/8)*180/np.pi
print('theta: ', theta2)

# nfullbound0_2 = 2.0*10**12 # in cm^-2  # I don't think this one is as reliable
# a = 0.246e-9
# thetabound0 = np.sqrt(np.sqrt(3)*nfullbound0_2*10**4*a**2/8)*180/np.pi
# print('theta bottom bound, based on half filling insulator: ', thetabound0)

nfullbound0_2 = 2.42*10**12 # in cm^-2  # reasonable alternative using half filling
a = 0.246e-9
thetabound0 = np.sqrt(np.sqrt(3)*nfullbound0_2*10**4*a**2/8)*180/np.pi
print('mid-range theta, based on half filling insulator: ', thetabound0)

nfullbound1_2 = 2.44*10**12 # in cm^-2
a = 0.246e-9
thetabound1 = np.sqrt(np.sqrt(3)*nfullbound1_2*10**4*a**2/8)*180/np.pi
print('theta top bound, based on full filling fan: ', thetabound1)

theta:  0.9946195187124093
mid-range theta, based on half filling insulator:  1.020236231925887
theta top bound, based on full filling fan:  1.0244434094942017


In [25]:
Cb = 0.0005809849079384244
prefactor = 2*Cb/1.602e-19/10**16
Vgatefull = 2.3/prefactor
V0 = -3.04

print('V0: ', V0)
print('Vgatefull: ', Vgatefull)
print('Prefactor: ', prefactor)

V0:  -3.04
Vgatefull:  3.1709945900957135
Prefactor:  0.7253244793238757


In [26]:
# plot in terms of nu to see if peaks line up
fig, ax = plt.subplots(1, 1, figsize=(10, 5))
normf = TwoSlopeNorm(vcenter=25, vmin=-1, vmax=60)
ax.pcolormesh(nd0_4041*4/2.3, B, Rxx4041d0/1000, cmap=cc.cm.diverging_bwr_20_95_c54, norm=normf)
ax.set_xticks(range(-5, 7))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [27]:
plt.figure()
plt.pcolormesh(Vb, B, Rxx4041d0/1000, vmin=0, vmax=60, cmap='Spectral_r')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Now looking at other D values

Use the equations

\begin{eqnarray}
n & = & \left[C_t V_t + C_b V_b\right]/e + n_0\\
D & = & \left[C_t V_t - C_b V_b\right]/2\epsilon_0
\end{eqnarray}

to map the gate voltages to n and D

In [28]:
def nD2928(Vb, Vt):
    Ct, Cb = 0.0028229350947619513, 0.0005809849079384244
    V0 = -3.07
    n0 = -2*Cb/1.602e-19*V0
    n = (Ct*Vt + Cb*Vb)/1.602e-19 + n0
    D = (Ct*Vt - Cb*Vb)/8.85e-12/2
    return n/10**16, D/10**9

def nD4041(Vb, Vt):
    Ct, Cb = 0.0028229350947619513, 0.0005809849079384244
    V0 = -3.1
    n0 = -2*Cb/1.602e-19*V0
    n = (Ct*Vt + Cb*Vb)/1.602e-19 + n0
    D = (Ct*Vt - Cb*Vb)/8.85e-12/2
    return n/10**16, D/10**9

In [29]:
nD2928(0, 0)

(2.2267461515242983, 0.0)

## 0 T

In [30]:
b, t = np.meshgrid(bg2d, tg2d)
n2928, D2928 = nD2928(b, t)
n4041, D4041 = nD4041(b, t)

In [31]:
fig, ax = plt.subplots(1, 2, figsize=(10, 5), sharey=True)
# dnorm = TwoSlopeNorm(vcenter=4, vmin=0, vmax=70)
ax[0].pcolormesh(n2928, D2928, R2928_2d/1000, cmap='inferno', norm=SymLogNorm(0.5, vmin=0, vmax=80))
ax[0].axis((-4, 4, -0.5, 0.6))
ax[1].pcolormesh(n4041, D4041, R4041_2d/1000, cmap='inferno', norm=SymLogNorm(0.5, vmin=0, vmax=80))
ax[1].axis((-4, 4, -0.5, 0.6))

ax[0].set_xlabel('n (10$^{12}$ cm$^{-2}$)')
ax[1].set_xlabel('n (10$^{12}$ cm$^{-2}$)')
ax[0].set_ylabel('D (V/nm)')

Do = nD2928(0, 2.9)[1]
Dg1 = nD2928(0, 1.3)[1]
Dg2 = nD2928(0, -1.2)[1]
Dr = nD2928(0, -2.2)[1]
ax[0].hlines([Do, Dg1, 0, Dg2, Dr], -4, 4, color=['orange', 'gray', 'blue', 'green', 'red'])
ax[1].hlines([Do, Dg1, 0, Dg2, Dr], -4, 4, color=['orange', 'gray', 'blue', 'green', 'red'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [32]:
slDrc = -0.2058088083627785

In [33]:
# Function for the orange line
def constant_D_orangeline_nsw(Vb):
    srframe.volt_p1(Vb)
    srframe.volt_p5(-slDrc*Vb + 2.9)
    
#Function for the grey line
def constant_D_greyline_nsw(Vb):
    srframe.volt_p1(Vb)
    srframe.volt_p5(-slDrc*Vb + 1.3)
    
#Function for the green line (blue)
def constant_D_greenline_nsw(Vb):
    srframe.volt_p1(Vb)
    srframe.volt_p5(-slDrc*Vb - 1.2)

#Zero field line
def zerofieldsw(Vb):
    srframe.volt_p1(Vb)
    srframe.volt_p5(-slDrc*Vb)

In [34]:

print('D field for each line (same for both electrodes)')
print('orange line: ', Do)
print('grey line: ', Dg1)
print('green line: ', Dg2)
print('red line: ', Dr)

D field for each line (same for both electrodes)
orange line:  0.46251478953726877
grey line:  0.20733421599946536
green line:  -0.19138543015335263
red line:  -0.35087328861447986


## Finite magnetic field 2D scans

In [32]:
d2d_8t = qc.load_data('data/2020-05-17/#003_M30_865_29-28__830_40-41_1nA_2dgatesweep_8T_16-27-29')
d2d_8t

DataSet:
   location = 'data/2020-05-17/#003_M30_865_29-28__830_40-41_1nA_2dgatesweep_8T_16-27-29'
   <Type>   | <array_id>          | <array.name> | <array.shape>
   Setpoint | srframe_volt_p5_set | volt_p5      | (193,)
   Setpoint | srframe_volt_p1_set | volt_p1      | (193, 273)
   Measured | lockin830_X         | X            | (193, 273)
   Measured | lockin830_Y         | Y            | (193, 273)
   Measured | lockin865_X         | X            | (193, 273)
   Measured | lockin865_Y         | Y            | (193, 273)

In [33]:
R2928_8t = npd.Rxxfromdata(d2d_8t, 1e-9, 'lockin865')[:, 11:]
R4041_8t = npd.Rxxfromdata(d2d_8t, 1e-9, 'lockin830')[:, 11:]

In [34]:
vt8t = d2d_8t.srframe_volt_p5_set[:]
vb8t = d2d_8t.srframe_volt_p1_set[0, 11:]

In [35]:
b, t = np.meshgrid(vb8t, vt8t)
n2928_8t, D2928_8t = nD2928(b, t)
n4041_8t, D4041_8t = nD4041(b, t)

In [36]:
fig, ax = plt.subplots(1, 2, figsize=(10, 5), sharey=True)
# dnorm = TwoSlopeNorm(vcenter=4, vmin=0, vmax=70)
ax[0].pcolormesh(n2928_8t, D2928_8t, R2928_8t/1000, cmap='inferno', norm=SymLogNorm(0.5, vmin=0, vmax=80))
ax[0].axis((-4, 4, -0.8, 0.75))
ax[1].pcolormesh(n4041_8t, D4041_8t, R4041_8t/1000, cmap='inferno', norm=SymLogNorm(0.5, vmin=0, vmax=80))
ax[1].axis((-4, 4, -0.8, 0.75))

ax[0].set_xlabel('n (10$^{12}$ cm$^{-2}$)')
ax[1].set_xlabel('n (10$^{12}$ cm$^{-2}$)')
ax[0].set_ylabel('D (V/nm)')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'D (V/nm)')

In [37]:
d2d_5t = qc.load_data('data/2020-05-18/#001_M30_865_29-28__830_40-41_1nA_2dgatesweep_5T_14-07-28')
d2d_5t

DataSet:
   location = 'data/2020-05-18/#001_M30_865_29-28__830_40-41_1nA_2dgatesweep_5T_14-07-28'
   <Type>   | <array_id>          | <array.name> | <array.shape>
   Setpoint | srframe_volt_p5_set | volt_p5      | (193,)
   Setpoint | srframe_volt_p1_set | volt_p1      | (193, 273)
   Measured | lockin830_X         | X            | (193, 273)
   Measured | lockin830_Y         | Y            | (193, 273)
   Measured | lockin865_X         | X            | (193, 273)
   Measured | lockin865_Y         | Y            | (193, 273)

In [38]:
nq = -60
R2928_5t = npd.Rxxfromdata(d2d_5t, 1e-9, 'lockin865')[:nq, 11:]
R4041_5t = npd.Rxxfromdata(d2d_5t, 1e-9, 'lockin830')[:nq, 11:]

In [39]:
vt5t = d2d_5t.srframe_volt_p5_set[:nq]
vb5t = d2d_5t.srframe_volt_p1_set[0, 11:]

In [40]:
b, t = np.meshgrid(vb5t, vt5t)
n2928_5t, D2928_5t = nD2928(b, t)
n4041_5t, D4041_5t = nD4041(b, t)

In [41]:
fig, ax = plt.subplots(1, 2, figsize=(10, 5), sharey=True)
# dnorm = TwoSlopeNorm(vcenter=4, vmin=0, vmax=70)
ax[0].pcolormesh(n2928_5t, D2928_5t, R2928_5t/1000, cmap='inferno', norm=SymLogNorm(0.5, vmin=0, vmax=80))
ax[0].axis((-4, 4, -0.8, 0.75))
ax[1].pcolormesh(n4041_5t, D4041_5t, R4041_5t/1000, cmap='inferno', norm=SymLogNorm(0.5, vmin=0, vmax=80))
ax[1].axis((-4, 4, -0.8, 0.75))

ax[0].set_xlabel('n (10$^{12}$ cm$^{-2}$)')
ax[1].set_xlabel('n (10$^{12}$ cm$^{-2}$)')
ax[0].set_ylabel('D (V/nm)')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'D (V/nm)')

# SC D dependence

In [35]:
shole = qc.load_data('data/2020-05-09/#007_M30_865_29-28__830_40-41_0p5nAexc_maxSChole_cosntant_n_D_sweep_AC0.5nAdvdi_baseT_b=-0.8mT_11-50-46')
shole

DataSet:
   location = 'data/2020-05-09/#007_M30_865_29-28__830_40-41_0p5nAexc_maxSChole_cosntant_n_D_sweep_AC0.5nAdvdi_baseT_b=-0.8mT_11-50-46'
   <Type>   | <array_id>               | <array.name>         | <array.shape>
   Setpoint | constant_n_maxholeSC_set | constant_n_maxholeSC | (45,)
   Setpoint | srdc_volt_set            | volt                 | (45, 161)
   Measured | lockin830_X              | X                    | (45, 161)
   Measured | lockin830_Y              | Y                    | (45, 161)
   Measured | lockin865_X              | X                    | (45, 161)
   Measured | lockin865_Y              | Y                    | (45, 161)

In [36]:
plt.figure()
plt.plot(shole.srdc_volt_set[0, :]*10, gaussian_filter(shole.lockin865_X[15, :]/0.5e-9, 0.7))
plt.xlabel('I (nA)')
plt.ylabel('dV/dI ($\Omega$)')
# plt.savefig(figfile + 'maxSChole.png', dpi=300, bbox_inches='tight')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'dV/dI ($\\Omega$)')

In [37]:
d4041 = qc.load_data('data/2020-05-11/#014_M30_865_29-28__830_40-41_0p5nAexc_constant_n_maxSC_40_41_Dsw_AC0.5nAdvdi_baseT_b=-0.8mT_18-39-28')
dvdi4041 = d4041.lockin830_X[:, :]/0.5e-9
Ivals = np.array(d4041.srdc_volt_set[0, :])*10
d4041

DataSet:
   location = 'data/2020-05-11/#014_M30_865_29-28__830_40-41_0p5nAexc_constant_n_maxSC_40_41_Dsw_AC0.5nAdvdi_baseT_b=-0.8mT_18-39-28'
   <Type>   | <array_id>                     | <array.name>               | <array.shape>
   Setpoint | constant_n_maxSC_40_41_Dsw_set | constant_n_maxSC_40_41_Dsw | (45,)
   Setpoint | srdc_volt_set                  | volt                       | (45, 161)
   Measured | lockin830_X                    | X                          | (45, 161)
   Measured | lockin830_Y                    | Y                          | (45, 161)
   Measured | lockin865_X                    | X                          | (45, 161)
   Measured | lockin865_Y                    | Y                          | (45, 161)

In [38]:
plt.figure()
plt.pcolormesh(Ivals, range(dvdi4041.shape[0]), dvdi4041)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [39]:
plt.figure()
plt.plot(Ivals, dvdi4041[0, :])
plt.plot(Ivals, dvdi4041[10, :])
plt.plot(Ivals, dvdi4041[40, :])
plt.savefig(figfile + '4041dvdi_constantn.png', dpi=300, bbox_inches='tight')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# T dependence

### Data gathering

In [40]:
dtempbase_orange = qc.load_data('data/2020-05-24/#006_M30_865_28-29_830_40-41_0p5nA_-0p8mT_orangeline_sweep_23-38-15')
dtemp_1_orange = qc.load_data('data/2020-05-25/#001_M30_865_28-29_830_40-41_0p5nA_gateTsweepto1K_orangeline_0p5nA_00-00-45')
dtemp_2_orange = qc.load_data('data/2020-05-25/#002_M30_865_28-29_830_40-41_0p5nA_gateTsweep1p1to1p4K_orangeline_0p5nA_06-27-37')
dtemp_3_orange = qc.load_data('data/2020-05-25/#003_M30_865_28-29_830_40-41_0p5nA_gateTsweep1p5to2K_orangeline_0p5nA_07-46-45')
dtemp_4_orange = qc.load_data('data/2020-05-25/#004_M30_865_28-29_830_40-41_0p5nA_gateTsweep2p1to10K_orangeline_0p5nA_11-57-46')
dtemp_5_orange = qc.load_data('data/2020-05-26/#001_M30_865_28-29_830_40-41_0p5nA_gateTsweep11to30K_orangeline_0p5nA_13-57-02')
dtemp_6_orange = qc.load_data('data/2020-05-26/#002_M30_865_28-29_830_40-41_0p5nA_gateTsweep31to40K_orangeline_0p5nA_20-10-58')

In [41]:
dtempbase_red = qc.load_data('data/2020-05-27/#002_M30_865_28-29_830_40-41_0.5nA_-0p8mT_redline_sweep_10-01-39')
dtemp_1_red = qc.load_data('data/2020-05-27/#003_M30_865_28-29_830_40-41_0p5nA_gateTsweepto1K_redline_0p5nA_10-26-44')
dtemp_2_red = qc.load_data('data/2020-05-27/#004_M30_865_28-29_830_40-41_0p5nA_gateTsweep1p1to1p4K_redline_0p5nA_16-38-35')
dtemp_3_red = qc.load_data('data/2020-05-27/#005_M30_865_28-29_830_40-41_0p5nA_gateTsweep1p5to2K_redline_0p5nA_17-54-18')
dtemp_4_red = qc.load_data('data/2020-05-27/#006_M30_865_28-29_830_40-41_0p5nA_gateTsweep2p1to10K_redline_0p5nA_20-12-53')
dtemp_5_red = qc.load_data('data/2020-05-28/#001_M30_865_28-29_830_40-41_0p5nA_gateTsweep11to30K_redline_0p5nA_21-13-37')
dtemp_6_red = qc.load_data('data/2020-05-29/#001_M30_865_28-29_830_40-41_0p5nA_gateTsweep31to40K_redline_0p5nA_03-02-32')

In [42]:
dtempbase_blue = qc.load_data('data/2020-06-04/#004_M30_865_28-29_830_40-41_0.5nA_-0p8mT_D0sweep_baseT_12-39-14')
dtemp_1_blue = qc.load_data('data/2020-06-04/#006_M30_865_28-29_830_40-41_0p5nA_gateTsweepto1K_D0_0p5nA_13-13-21')
dtemp_2_blue = qc.load_data('data/2020-06-04/#007_M30_865_28-29_830_40-41_0p5nA_gateTsweep1p1to1p4K_D0_0p5nA_19-54-05')
dtemp_3_blue = qc.load_data('data/2020-06-04/#008_M30_865_28-29_830_40-41_0p5nA_gateTsweep1p5to2K_D0_0p5nA_21-15-39')
dtemp_4_blue = qc.load_data('data/2020-06-04/#009_M30_865_28-29_830_40-41_0p5nA_gateTsweep2p1to10K_D0_0p5nA_23-32-58')
dtemp_5_blue = qc.load_data('data/2020-06-06/#001_M30_865_28-29_830_40-41_0p5nA_gateTsweep11to30K_D0_0p5nA_02-10-41')
dtemp_6_blue = qc.load_data('data/2020-06-06/#002_M30_865_28-29_830_40-41_0p5nA_gateTsweep31to40K_D0_0p5nA_08-24-24')

In [43]:
temps = np.concatenate(([0.025], dtemp_1_orange.triton_pid_setpoint_set, dtemp_2_orange.triton_pid_setpoint_set[:], dtemp_3_orange.triton_pid_setpoint_set, dtemp_4_orange.triton_pid_setpoint_set))
temps_high = np.concatenate(([0.025], dtemp_1_orange.triton_pid_setpoint_set, dtemp_2_orange.triton_pid_setpoint_set[:], dtemp_3_orange.triton_pid_setpoint_set, dtemp_4_orange.triton_pid_setpoint_set, dtemp_5_orange.triton_pid_setpoint_set, dtemp_6_orange.triton_pid_setpoint_set))
tgate_orange = dtempbase_orange.constant_D_orangeline_nsw_set[:]
R2829_orange = np.vstack((npd.Rxxfromdata(dtempbase_orange , 0.5e-9), npd.Rxxfromdata(dtemp_1_orange, 0.5e-9), npd.Rxxfromdata(dtemp_2_orange, 0.5e-9), npd.Rxxfromdata(dtemp_3_orange, 0.5e-9), npd.Rxxfromdata(dtemp_4_orange, 0.5e-9)))
R2829_orange_high = np.vstack((npd.Rxxfromdata(dtempbase_orange , 0.5e-9), npd.Rxxfromdata(dtemp_1_orange, 0.5e-9), npd.Rxxfromdata(dtemp_2_orange, 0.5e-9), npd.Rxxfromdata(dtemp_3_orange, 0.5e-9), npd.Rxxfromdata(dtemp_4_orange, 0.5e-9), npd.Rxxfromdata(dtemp_5_orange, 0.5e-9), npd.Rxxfromdata(dtemp_6_orange, 0.5e-9)))
R4041_orange = np.vstack((npd.Rxxfromdata(dtempbase_orange , 0.5e-9, 'lockin830'), npd.Rxxfromdata(dtemp_1_orange, 0.5e-9, 'lockin830'), npd.Rxxfromdata(dtemp_2_orange, 0.5e-9, 'lockin830'), npd.Rxxfromdata(dtemp_3_orange, 0.5e-9, 'lockin830'), npd.Rxxfromdata(dtemp_4_orange, 0.5e-9, 'lockin830')))
R4041_orange_high = np.vstack((npd.Rxxfromdata(dtempbase_orange , 0.5e-9, 'lockin830'), npd.Rxxfromdata(dtemp_1_orange, 0.5e-9, 'lockin830'), npd.Rxxfromdata(dtemp_2_orange, 0.5e-9, 'lockin830'), npd.Rxxfromdata(dtemp_3_orange, 0.5e-9, 'lockin830'), npd.Rxxfromdata(dtemp_4_orange, 0.5e-9, 'lockin830'), npd.Rxxfromdata(dtemp_5_orange, 0.5e-9, 'lockin830'), npd.Rxxfromdata(dtemp_6_orange, 0.5e-9, 'lockin830')))

In [44]:
temps = np.concatenate(([0.025], dtemp_1_red.triton_pid_setpoint_set, dtemp_2_red.triton_pid_setpoint_set[:], dtemp_3_red.triton_pid_setpoint_set, dtemp_4_red.triton_pid_setpoint_set[:]))
tgate_red = dtempbase_red.constant_D_redline_nsw_set[:]
R2829_red = np.vstack((npd.Rxxfromdata(dtempbase_red , 0.5e-9), npd.Rxxfromdata(dtemp_1_red, 0.5e-9), npd.Rxxfromdata(dtemp_2_red, 0.5e-9), npd.Rxxfromdata(dtemp_3_red, 0.5e-9), npd.Rxxfromdata(dtemp_4_red, 0.5e-9)))
R2829_red_high = np.vstack((npd.Rxxfromdata(dtempbase_red , 0.5e-9), npd.Rxxfromdata(dtemp_1_red, 0.5e-9), npd.Rxxfromdata(dtemp_2_red, 0.5e-9), npd.Rxxfromdata(dtemp_3_red, 0.5e-9), npd.Rxxfromdata(dtemp_4_red, 0.5e-9), npd.Rxxfromdata(dtemp_5_red, 0.5e-9), npd.Rxxfromdata(dtemp_6_red, 0.5e-9)))
R4041_red = np.vstack((npd.Rxxfromdata(dtempbase_red , 0.5e-9, 'lockin830'), npd.Rxxfromdata(dtemp_1_red, 0.5e-9, 'lockin830'), npd.Rxxfromdata(dtemp_2_red, 0.5e-9, 'lockin830'), npd.Rxxfromdata(dtemp_3_red, 0.5e-9, 'lockin830'), npd.Rxxfromdata(dtemp_4_red, 0.5e-9, 'lockin830')))
R4041_red_high = np.vstack((npd.Rxxfromdata(dtempbase_red , 0.5e-9, 'lockin830'), npd.Rxxfromdata(dtemp_1_red, 0.5e-9, 'lockin830'), npd.Rxxfromdata(dtemp_2_red, 0.5e-9, 'lockin830'), npd.Rxxfromdata(dtemp_3_red, 0.5e-9, 'lockin830'), npd.Rxxfromdata(dtemp_4_red, 0.5e-9, 'lockin830'), npd.Rxxfromdata(dtemp_5_red, 0.5e-9, 'lockin830'), npd.Rxxfromdata(dtemp_6_red, 0.5e-9, 'lockin830')))

In [45]:
temps = np.concatenate(([0.025], dtemp_1_blue.triton_pid_setpoint_set, dtemp_2_blue.triton_pid_setpoint_set[:], dtemp_3_blue.triton_pid_setpoint_set, dtemp_4_blue.triton_pid_setpoint_set))
tgate_blue = dtempbase_blue.zeroDparam_set[:]
R2829_blue = np.vstack((npd.Rxxfromdata(dtempbase_blue , 0.5e-9), npd.Rxxfromdata(dtemp_1_blue, 0.5e-9), npd.Rxxfromdata(dtemp_2_blue, 0.5e-9), npd.Rxxfromdata(dtemp_3_blue, 0.5e-9), npd.Rxxfromdata(dtemp_4_blue, 0.5e-9)))
R2829_blue_high = np.vstack((npd.Rxxfromdata(dtempbase_blue , 0.5e-9), npd.Rxxfromdata(dtemp_1_blue, 0.5e-9), npd.Rxxfromdata(dtemp_2_blue, 0.5e-9), npd.Rxxfromdata(dtemp_3_blue, 0.5e-9), npd.Rxxfromdata(dtemp_4_blue, 0.5e-9), npd.Rxxfromdata(dtemp_5_blue, 0.5e-9), npd.Rxxfromdata(dtemp_6_blue, 0.5e-9)))
R4041_blue = np.vstack((npd.Rxxfromdata(dtempbase_blue , 0.5e-9, 'lockin830'), npd.Rxxfromdata(dtemp_1_blue, 0.5e-9, 'lockin830'), npd.Rxxfromdata(dtemp_2_blue, 0.5e-9, 'lockin830'), npd.Rxxfromdata(dtemp_3_blue, 0.5e-9, 'lockin830'), npd.Rxxfromdata(dtemp_4_blue, 0.5e-9, 'lockin830')))
R4041_blue_high = np.vstack((npd.Rxxfromdata(dtempbase_blue , 0.5e-9, 'lockin830'), npd.Rxxfromdata(dtemp_1_blue, 0.5e-9, 'lockin830'), npd.Rxxfromdata(dtemp_2_blue, 0.5e-9, 'lockin830'), npd.Rxxfromdata(dtemp_3_blue, 0.5e-9, 'lockin830'), npd.Rxxfromdata(dtemp_4_blue, 0.5e-9, 'lockin830'), npd.Rxxfromdata(dtemp_5_blue, 0.5e-9, 'lockin830'), npd.Rxxfromdata(dtemp_6_blue, 0.5e-9, 'lockin830')))

In [46]:
R2829_orangecorr = np.copy(R2829_orange)
R2829_orangecorr[R2829_orange <= 0] = np.min(np.abs(R2829_orange[R2829_orange > 0]))
R4041_orangecorr = np.copy(R4041_orange)
R4041_orangecorr[R4041_orange <= 0] = np.min(np.abs(R4041_orange[R4041_orange > 0]))

In [47]:
R2829_redcorr = np.copy(R2829_red)
R2829_redcorr[R2829_red <= 0] = np.min(np.abs(R2829_red[R2829_red > 0]))
R4041_redcorr = np.copy(R4041_red)
R4041_redcorr[R4041_red <= 0] = np.min(np.abs(R4041_red[R4041_red > 0]))

In [48]:
R2829_bluecorr = np.copy(R2829_blue)
R2829_bluecorr[R2829_blue <= 0] = np.min(np.abs(R2829_blue[R2829_blue > 0]))
R4041_bluecorr = np.copy(R4041_blue)
R4041_bluecorr[R4041_blue <= 0] = np.min(np.abs(R4041_blue[R4041_blue > 0]))

In [49]:
tgate_orange_corr = tgate_orange[2:]  # need to correct for a gate offset at baseT
R2829_orangecorr = R2829_orangecorr[:, 2:]
R2829_orangecorr[0, :] = R2829_orange [0, :-2]
R2829_orangecorr[1, :] = R2829_orange [1, 1:-1]
R2829_orangecorr[31, :] = R2829_orange [31, 1:-1]
R2829_orangecorr[32, :] = R2829_orange [32, 1:-1]

In [50]:
tgate_orange_corr = tgate_orange[2:]  # need to correct for a gate offset at baseT
R4041_orangecorr = R4041_orangecorr[:, 2:]
R4041_orangecorr[0, :] = R4041_orange [0, :-2]
R4041_orangecorr[1, :] = R4041_orange [1, 1:-1]
R4041_orangecorr[31, :] = R4041_orange [31, 1:-1]
R4041_orangecorr[32, :] = R4041_orange [32, 1:-1]

In [51]:
tgate_red_corr = tgate_red[1:]  # need to correct for a gate offset at baseT
R2829_redcorr = R2829_redcorr[:, 1:]
R2829_redcorr[1, :] = R2829_red [1, :-1]
R2829_redcorr[2, :] = R2829_red [2, :-1]

In [52]:
tgate_red_corr = tgate_red[1:]  # need to correct for a gate offset at baseT
R4041_redcorr = R4041_redcorr[:, 1:]
R4041_redcorr[1, :] = R4041_red [1, :-1]
R4041_redcorr[2, :] = R4041_red [2, :-1]

### Smaller-angle area with gapped insulator and electron and hole superconductivity

In [53]:
ft = interp2d(tgate_orange_corr, temps, R2829_orangecorr, kind='cubic')

In [54]:
tgate_orange_interp = np.linspace(-13.58, -6.8, 679)
temps_interp = np.linspace(0.025, 2, 80)
R2829_orangecorr_interp = ft(tgate_orange_interp, temps_interp)

In [55]:
ft3 = interp2d(tgate_red_corr, temps, R2829_redcorr, kind='cubic')

In [56]:
tgate_red_interp = np.linspace(-0.5, 6, 651)
temps_interp = np.linspace(0.025, 2, 80)
R2829_redcorr_interp = ft3(tgate_red_interp, temps_interp)

In [57]:
ft5 = interp2d(tgate_blue, temps, R2829_bluecorr, kind='cubic')

In [58]:
tgate_blue_interp = np.linspace(-6.5, 0.6, 711)
temps_interp = np.linspace(0.025, 2, 80)
R2829_bluecorr_interp = ft5(tgate_blue_interp, temps_interp)

In [59]:
fig, ax = plt.subplots(3, 1, figsize=(10, 14))
coff = colors.SymLogNorm(750, vmin=0, vmax=2e4)

p1 = ax[0].pcolormesh(tgate_orange_interp, temps_interp, R2829_orangecorr_interp, norm=coff, cmap='seismic')
ax[0].set_ylabel('T (K)')
ax[0].set_xlim(-13.6, -6.8)
ax[0].set_title('28-29 orange line D = 0.925 V/nm')
fig.colorbar(p1, ax=ax[0], label='R$_{xx}$ (k$\Omega$)')

p2 = ax[1].pcolormesh(tgate_blue_interp, temps_interp, R2829_bluecorr_interp, norm=coff, cmap='seismic')
ax[1].set_ylabel('T (K)')
ax[1].set_xlim(-6.55, 0.25)
ax[1].set_title('blue line D=0 V/nm')
fig.colorbar(p2, ax=ax[1], label='R$_{xx}$ (k$\Omega$)')

p3 = ax[2].pcolormesh(tgate_red_interp, temps_interp, R2829_redcorr_interp, norm=coff, cmap='seismic')
ax[2].set_ylabel('T (K)')
ax[2].set_xlabel('Vg (V)')
ax[2].set_xlim(-1.2, 5.6)
ax[2].set_title('red line D = -0.7 V/nm')
fig.colorbar(p3, ax=ax[2], label='R$_{xx}$ (k$\Omega$)')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [60]:
fig, ax = plt.subplots(3, 1, figsize=(10, 14))
coff = colors.SymLogNorm(1000, vmin=0, vmax=2e4)

p1 = ax[0].pcolormesh(tgate_orange_corr, temps, R2829_orangecorr, norm=coff, cmap='seismic')
ax[0].set_ylabel('T (K)')
ax[0].set_xlim(-13.6, -6.8)
ax[0].set_title('28-29 orange line D = 0.925 V/nm')
fig.colorbar(p1, ax=ax[0], label='R$_{xx}$ (k$\Omega$)')

p2 = ax[1].pcolormesh(tgate_blue, temps, R2829_bluecorr, norm=coff, cmap='seismic')
ax[1].set_ylabel('T (K)')
ax[1].set_xlim(-6.55, 0.25)
ax[1].set_title('blue line D=0 V/nm')
fig.colorbar(p2, ax=ax[1], label='R$_{xx}$ (k$\Omega$)')

p3 = ax[2].pcolormesh(tgate_red_corr, temps, R2829_redcorr, norm=coff, cmap='seismic')
ax[2].set_ylabel('T (K)')
ax[2].set_xlabel('Vg (V)')
ax[2].set_xlim(-1.2, 5.6)
ax[2].set_title('red line D = -0.7 V/nm')
fig.colorbar(p3, ax=ax[2], label='R$_{xx}$ (k$\Omega$)')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [61]:
fig, ax = plt.subplots(3, 1, figsize=(10, 14))
coff = colors.SymLogNorm(1500, vmin=0, vmax=3e4)

p1 = ax[0].pcolormesh(tgate_orange, temps_high, R2829_orange_high, norm=coff, cmap='seismic')
ax[0].set_ylabel('T (K)')
ax[0].set_xlim(-13.6, -6.8)
ax[0].set_title('28-29 orange line D = 0.925 V/nm')
fig.colorbar(p1, ax=ax[0], label='R$_{xx}$ (k$\Omega$)')

p2 = ax[1].pcolormesh(tgate_blue, temps_high, R2829_blue_high, norm=coff, cmap='seismic')
ax[1].set_ylabel('T (K)')
ax[1].set_xlim(-6.55, 0.25)
ax[1].set_title('blue line D = 0 V/nm')
fig.colorbar(p2, ax=ax[1], label='R$_{xx}$ (k$\Omega$)')

p3 = ax[2].pcolormesh(tgate_red, temps_high, R2829_red_high, norm=coff, cmap='seismic')
ax[2].set_ylabel('T (K)')
ax[2].set_xlabel('Vg (V)')
ax[2].set_xlim(-1.2, 5.6)
ax[2].set_title('red line D = -0.7 V/nm')
fig.colorbar(p3, ax=ax[2], label='R$_{xx}$ (k$\Omega$)')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Electrodes 40-41: larger angle with likely FM and more correlated features at integer filling

In [62]:
ft2 = interp2d(tgate_orange_corr, temps, R4041_orangecorr, kind='cubic')

In [63]:
tgate_orange_interp = np.linspace(-13.58, -6.8, 679)
temps_interp = np.linspace(0.025, 2, 80)
R4041_orangecorr_interp = ft2(tgate_orange_interp, temps_interp)

In [64]:
ft4 = interp2d(tgate_red_corr, temps, R4041_redcorr, kind='cubic')

In [65]:
tgate_red_interp = np.linspace(-0.5, 6, 651)
temps_interp = np.linspace(0.025, 2, 80)
R4041_redcorr_interp = ft4(tgate_red_interp, temps_interp)

In [66]:
ft6 = interp2d(tgate_blue, temps, R4041_bluecorr, kind='cubic')

In [67]:
tgate_blue_interp = np.linspace(-6.5, 0.6, 711)
temps_interp = np.linspace(0.025, 2, 80)
R4041_bluecorr_interp = ft6(tgate_blue_interp, temps_interp)

In [68]:
fig, ax = plt.subplots(3, 1, figsize=(10, 14))
coff = colors.SymLogNorm(4000, vmin=0, vmax=5e4)

p1 = ax[0].pcolormesh(tgate_orange_interp, temps_interp, R4041_orangecorr_interp, norm=coff, cmap='seismic')
ax[0].set_ylabel('T (K)')
ax[0].set_xlim(-13.6, -6.8)
ax[0].set_title('40-41 orange line D = 0.925 V/nm')
fig.colorbar(p1, ax=ax[0], label='R$_{xx}$ (k$\Omega$)')

p2 = ax[1].pcolormesh(tgate_blue_interp, temps_interp, R4041_bluecorr_interp, norm=coff, cmap='seismic')
ax[1].set_ylabel('T (K)')
ax[1].set_xlim(-6.55, 0.25)
ax[1].set_title('blue line D=0 V/nm')
fig.colorbar(p2, ax=ax[1], label='R$_{xx}$ (k$\Omega$)')

p3 = ax[2].pcolormesh(tgate_red_interp, temps_interp, R4041_redcorr_interp, norm=coff, cmap='seismic')
ax[2].set_ylabel('T (K)')
ax[2].set_xlabel('Vg (V)')
ax[2].set_xlim(-1.2, 5.6)
ax[2].set_title('40-41 red line D = -0.7 V/nm')
fig.colorbar(p3, ax=ax[2], label='R$_{xx}$ (k$\Omega$)')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [69]:
fig, ax = plt.subplots(3, 1, figsize=(10, 14))
coff = colors.SymLogNorm(4000, vmin=0, vmax=5e4)

p1 = ax[0].pcolormesh(tgate_orange_corr, temps, R4041_orangecorr, norm=coff, cmap='seismic')
ax[0].set_ylabel('T (K)')
ax[0].set_xlim(-13.6, -6.8)
ax[0].set_title('40-41 orange line D = 0.925 V/nm')
fig.colorbar(p1, ax=ax[0], label='R$_{xx}$ (k$\Omega$)')

p2 = ax[1].pcolormesh(tgate_blue, temps, R4041_bluecorr, norm=coff, cmap='seismic')
ax[1].set_ylabel('T (K)')
ax[1].set_xlim(-6.55, 0.25)
ax[1].set_title('blue line D=0 V/nm')
fig.colorbar(p2, ax=ax[1], label='R$_{xx}$ (k$\Omega$)')

p3 = ax[2].pcolormesh(tgate_red_corr, temps, R4041_redcorr, norm=coff, cmap='seismic')
ax[2].set_ylabel('T (K)')
ax[2].set_xlabel('Vg (V)')
ax[2].set_xlim(-1.2, 5.6)
ax[2].set_title('red line D = -0.7 V/nm')
fig.colorbar(p3, ax=ax[2], label='R$_{xx}$ (k$\Omega$)')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [70]:
fig, ax = plt.subplots(3, 1, figsize=(10, 14))
coff = colors.SymLogNorm(5000, vmin=0, vmax=5e4)

p1 = ax[0].pcolormesh(tgate_orange, temps_high, R4041_orange_high, norm=coff, cmap='seismic')
ax[0].set_ylabel('T (K)')
ax[0].set_xlim(-13.6, -6.8)
ax[0].set_title('4041 orange line D = 0.925 V/nm')
fig.colorbar(p1, ax=ax[0], label='R$_{xx}$ (k$\Omega$)')

p2 = ax[1].pcolormesh(tgate_blue, temps_high, R4041_blue_high, norm=coff, cmap='seismic')
ax[1].set_ylabel('T (K)')
ax[1].set_xlim(-6.55, 0.25)
ax[1].set_title('blue line D = 0 V/nm')
fig.colorbar(p2, ax=ax[1], label='R$_{xx}$ (k$\Omega$)')

p3 = ax[2].pcolormesh(tgate_red, temps_high, R4041_red_high, norm=coff, cmap='seismic')
ax[2].set_ylabel('T (K)')
ax[2].set_xlabel('Vg (V)')
ax[2].set_xlim(-1.2, 5.6)
ax[2].set_title('red line D = -0.7 V/nm')
fig.colorbar(p3, ax=ax[2], label='R$_{xx}$ (k$\Omega$)')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Now plot with respect to filling factor

Raw data

In [71]:
V0 =  -3.04
Vgatefull =  3.1709945900957135
nugate_blue = np.interp(tgate_blue, [V0-Vgatefull*6/4, V0+Vgatefull*6/4], [-6, 6])

In [72]:
fig, ax = plt.subplots(1, 1, figsize=(9, 4))
p = ax.pcolormesh(nugate_blue, temps, R4041_bluecorr, norm=coff, cmap='seismic')
ax.set_ylabel('T (K)')
# ax.set_xlim(-6.55, 0.25)
ax.set_ylim(0.025, 2)
ax.set_title('blue line D=0 V/nm')
fig.colorbar(p, ax=ax, label='R$_{xx}$ (k$\Omega$)')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [73]:
fig, ax = plt.subplots(1, 1, figsize=(9, 4))
norm = TwoSlopeNorm(7.5, 0, 40)
p = ax.pcolormesh(nugate_blue, temps_high, R4041_blue_high/1000, norm=norm, cmap=cc.cm.diverging_bwr_20_95_c54)
ax.set_ylabel('T (K)')
ax.set_xlabel('$\\nu$')
# ax.set_title('blue line D = 0 V/nm')
fig.colorbar(p, ax=ax, label='R$_{xx}$ (k$\Omega$)')
ax.set_ylim(0.025, 20)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(0.025, 20.0)

In [74]:
fig, ax = plt.subplots(1, 1, figsize=(2, 4))
norm = TwoSlopeNorm(7.5, 0, 40)
p = ax.pcolormesh(nugate_blue, temps, R4041_bluecorr/1000, norm=norm, cmap=cc.cm.diverging_bwr_20_95_c54)
ax.set_ylabel('T (K)')
ax.set_xlabel('$\\nu$')
ax.axis((0.5, 1.3, 0.025, 8))
# ax.set_title('blue line D=0 V/nm')
# fig.colorbar(p, ax=ax, label='R$_{xx}$ (k$\Omega$)')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(0.5, 1.3, 0.025, 8.0)

In [75]:
fig, ax = plt.subplots(1, 1, figsize=(9, 4))
norm = TwoSlopeNorm(8, 0, 40)
p = ax.pcolormesh(nugate_blue, temps, R4041_bluecorr/1000, norm=norm, cmap=cc.cm.diverging_bwr_20_95_c54)
ax.set_ylabel('T (K)')
ax.set_xlabel('$\\nu$')
# ax.set_title('blue line D=0 V/nm')
fig.colorbar(p, ax=ax, label='R$_{xx}$ (k$\Omega$)')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Red line (negative D)

In [76]:
V0red = 2.32
nugate_red = np.interp(tgate_red_corr, [V0red - Vgatefull*6/4, V0red + Vgatefull*6/4], [-6, 6])

In [77]:
fig, ax = plt.subplots(1, 1, figsize=(9, 4))
p = ax.pcolormesh(nugate_red, temps, R4041_redcorr, norm=coff, cmap='seismic')
ax.set_ylabel('T (K)')
ax.set_xlabel('$\\nu$')
# ax.set_xlim(-1.2, 5.6)
ax.set_title('red line D = -0.7 V/nm')
fig.colorbar(p, ax=ax, label='R$_{xx}$ (k$\Omega$)')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Orange line (postive D)

In [78]:
V0_orange = -10.1
nugate_orange = np.interp(tgate_orange_corr, [V0_orange - 6/4*Vgatefull, V0_orange + 6/4*Vgatefull], [-6, 6])

In [79]:
fig, ax = plt.subplots(1, 1, figsize=(9, 4))
p = ax.pcolormesh(nugate_orange, temps, R4041_orangecorr, norm=coff, cmap='seismic')
ax.set_ylabel('T (K)')
# ax.set_xlim(-13.6, -6.8)
ax.set_title('40-41 orange line D = 0.925 V/nm')
fig.colorbar(p, ax=ax, label='R$_{xx}$ (k$\Omega$)')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### SC Tc determination

0 D field

In [80]:
plt.figure()
plt.plot(temps, R4041_bluecorr[:, 153]/1000)
print('nu = ', nugate_blue[153])

plt.axis((-0.2, 2.6, 0, 5.5))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

nu =  -2.434567382774052


(-0.2, 2.6, 0.0, 5.5)

Red line

In [81]:
plt.figure()
plt.plot(temps, R4041_redcorr[:, 555]/1000)


print('nu = ', nugate_red[555])

plt.axis((-0.2, 2.6, -0.1, 5.5))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

nu =  -2.371495688919802


(-0.2, 2.6, -0.1, 5.5)

Orange line

In [82]:
plt.figure()

plt.plot(temps, R4041_orangecorr[:, 150]/1000)

print('nu = ', nugate_orange[150])

plt.axis((-0.2, 2.6, -0.1, 5.5))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

nu =  -2.497639076628303


(-0.2, 2.6, -0.1, 5.5)

# WAL D dependence

## D = 0

In [55]:
def nzero2928(Vb):
    # the density (in 10^12 cm^-2) for electrodes 29-28, on the the zero-field line, given Vb
    return nD2928(Vb, -slDrc*Vb)[0]

def nzero4041(Vb):
    # the density (in 10^12 cm^-2) for electrodes 29-28, on the the zero-field line, given Vb
    return nD4041(Vb, -slDrc*Vb)[0]

In [67]:
d0sweep = qc.load_data('data/2020-06-06/#003_M30_865_28-29_830_40-41_1nA_0T_D0sweep_baseT_18-16-11')
Vb0sweep = d0sweep.zeroDparam_set[:]
n02928 = nzero2928(Vb0sweep)
n04041 = nzero4041(Vb0sweep)
Rxx0sweep2928 = npd.Rxxfromdata(d0sweep, 1e-9, 'lockin865')
Rxx0sweep4041 = npd.Rxxfromdata(d0sweep, 1e-9, 'lockin830')

In [76]:
fig, ax = plt.subplots(1, 2, figsize=(10, 3.5))

ax[0].semilogy(n02928, Rxx0sweep2928)
ax[0].axis([-2.7, 2.8, 10, 4e5])
ax[0].set_xlabel('n (10$^{12}$ cm$^{-2}$)')
ax[0].set_ylabel('R$_{xx}$ ($\Omega$)')

ax[1].semilogy(n04041, Rxx0sweep4041)
ax[1].axis([-2.7, 2.8, 200, 2e5])
ax[1].set_xlabel('n (10$^{12}$ cm$^{-2}$)')

plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Course scan

In [523]:
zeroDcoursef = sorted(glob.glob('data/2020-06-06/#*/'))[3:] + sorted(glob.glob('data/2020-06-07/#*/'))[:79]
len(zeroDcoursef)

126

In [524]:
esqh = 1.602e-19**2/6.626e-34

In [525]:
barr = np.linspace(-20, 20, 201)
Vbvals = np.linspace(-4.5, -2, 126)
nvals2928 = nzero2928(Vbvals)
nvals4041 = nzero4041(Vbvals)

In [526]:
# raw Resistance data placed into an array
walarr1_865 = np.zeros((len(nvals2928), len(barr)))
for i, f in enumerate(zeroDcoursef):
    d = qc.load_data(f[:-1])
    dlast = np.where(np.isnan(d.triton_field))[0][0]
    sweep = np.interp(barr/1000, d.triton_field[:dlast], d.lockin865_X[:dlast]/10e-9)
    walarr1_865[i, :] = sweep

# form an array of change in conductance from the zero point
b01 = -0.6  # the offset of the field zero point
pind = npd.val_to_index([b01], barr)[0]
dG1_865 = np.zeros(walarr1_865.shape)
for i in range(len(nvals2928)):
    dG1_865[i, :] = 1/walarr1_865[i, :]/esqh - 1/walarr1_865[i, pind]/esqh

-0.60 is not a value in the array
Plotted -0.60 instead


In [527]:
plt.figure()
plt.pcolormesh(barr-b01, nvals2928, dG1_865, vmin=-1, vmax=0.1)
plt.colorbar()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Holes (fine scale) base T

In [528]:
barr = np.linspace(-25, 25, 321)

In [529]:
zeroDholef = sorted(glob.glob('data/2020-06-07/#*/'))[79:120]
len(zeroDholef)

41

In [530]:
Vbvals = np.linspace(-3.5, -3.1, 41)
nvals2928_2 = nzero2928(Vbvals)
nvals4041_2 = nzero4041(Vbvals)

In [531]:
# raw Resistance data placed into an array
walarr2_865 = np.zeros((len(nvals2928_2), len(barr)))
for i, f in enumerate(zeroDholef):
    d = qc.load_data(f[:-1])
    dlast = np.where(np.isnan(d.triton_field))[0][0]
    sweep = np.interp(barr/1000, d.triton_field[:dlast], d.lockin865_X[:dlast]/10e-9)
    walarr2_865[i, :] = sweep

# form an array of change in conductance from the zero point
b02 = -0.31  # the offset of the field zero point
pind = npd.val_to_index([b02], barr)[0]
dG2_865 = np.zeros(walarr2_865.shape)
for i in range(walarr2_865.shape[0]):
    dG2_865[i, :] = 1/walarr2_865[i, :]/esqh - 1/walarr2_865[i, pind]/esqh

-0.31 is not a value in the array
Plotted -0.31 instead


In [532]:
plt.figure()
plt.pcolormesh(barr-b02, nvals2928_2, dG2_865, vmin=-1, vmax=0.1)
plt.colorbar()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [533]:
plt.figure()
plt.plot(barr-b02, np.average(dG2_865[:], axis=0), label='all')
plt.plot(barr-b02, np.average(dG2_865[:20], axis=0), label='sweeps 0 to 20')
plt.plot(barr-b02, np.average(dG2_865[20:40], axis=0), label='sweeps 20 to 40')
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Holes 300 mK

In [534]:
zeroDholef300 = sorted(glob.glob('data/2020-06-11/#*/'))[38:59]
zeroDholef300

['data/2020-06-11/#039_M30_865_28-29_830_40-41_300mK_WALfieldsw_D0_Vbg-3.50_10nA_10-18-39/',
 'data/2020-06-11/#040_M30_865_28-29_830_40-41_300mK_WALfieldsw_D0_Vbg-3.48_10nA_10-29-46/',
 'data/2020-06-11/#041_M30_865_28-29_830_40-41_300mK_WALfieldsw_D0_Vbg-3.46_10nA_10-40-51/',
 'data/2020-06-11/#042_M30_865_28-29_830_40-41_300mK_WALfieldsw_D0_Vbg-3.44_10nA_10-51-56/',
 'data/2020-06-11/#043_M30_865_28-29_830_40-41_300mK_WALfieldsw_D0_Vbg-3.42_10nA_11-03-02/',
 'data/2020-06-11/#044_M30_865_28-29_830_40-41_300mK_WALfieldsw_D0_Vbg-3.40_10nA_11-14-08/',
 'data/2020-06-11/#045_M30_865_28-29_830_40-41_300mK_WALfieldsw_D0_Vbg-3.38_10nA_11-25-15/',
 'data/2020-06-11/#046_M30_865_28-29_830_40-41_300mK_WALfieldsw_D0_Vbg-3.36_10nA_11-36-22/',
 'data/2020-06-11/#047_M30_865_28-29_830_40-41_300mK_WALfieldsw_D0_Vbg-3.34_10nA_11-47-30/',
 'data/2020-06-11/#048_M30_865_28-29_830_40-41_300mK_WALfieldsw_D0_Vbg-3.32_10nA_11-58-35/',
 'data/2020-06-11/#049_M30_865_28-29_830_40-41_300mK_WALfieldsw_D0_Vbg

In [535]:
Vbvals = np.linspace(-3.5, -3.1, 21)
nvals2928_300 = nzero2928(Vbvals)
nvals4041_300 = nzero4041(Vbvals)

In [536]:
# raw Resistance data placed into an array
walarr300_865 = np.zeros((len(nvals2928_300), len(barr)))
for i, f in enumerate(zeroDholef300):
    d = qc.load_data(f[:-1])
    dlast = np.where(np.isnan(d.triton_field))[0][0]
    sweep = np.interp(barr/1000, d.triton_field[:dlast], d.lockin865_X[:dlast]/10e-9)
    walarr300_865[i, :] = sweep

# form an array of change in conductance from the zero point
b02 = -0.31  # the offset of the field zero point
pind = npd.val_to_index([b02], barr)[0]
dG300_865 = np.zeros(walarr300_865.shape)
for i in range(walarr300_865.shape[0]):
    dG300_865[i, :] = 1/walarr300_865[i, :]/esqh - 1/walarr300_865[i, pind]/esqh

-0.31 is not a value in the array
Plotted -0.31 instead


In [537]:
plt.figure()
plt.pcolormesh(barr-b02, nvals2928_300, dG300_865, vmin=-1, vmax=0.1)
plt.colorbar()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [538]:
plt.figure()
plt.plot(barr-b02, np.average(dG300_865[:], axis=0))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Holes 600 mK

In [539]:
zeroDholef600 = sorted(glob.glob('data/2020-06-12/#*/'))[68:89]

In [540]:
Vbvals = np.linspace(-3.5, -3.1, 21)
nvals2928_600 = nzero2928(Vbvals)
nvals4041_600 = nzero4041(Vbvals)

In [541]:
# raw Resistance data placed into an array
walarr600_865 = np.zeros((len(nvals2928_600), len(barr)))
for i, f in enumerate(zeroDholef600):
    d = qc.load_data(f[:-1])
    dlast = np.where(np.isnan(d.triton_field))[0][0]
    sweep = np.interp(barr/1000, d.triton_field[:dlast], d.lockin865_X[:dlast]/10e-9)
    walarr600_865[i, :] = sweep

# form an array of change in conductance from the zero point
b02 = -0.31  # the offset of the field zero point
pind = npd.val_to_index([b02], barr)[0]
dG600_865 = np.zeros(walarr600_865.shape)
for i in range(walarr300_865.shape[0]):
    dG600_865[i, :] = 1/walarr600_865[i, :]/esqh - 1/walarr600_865[i, pind]/esqh

-0.31 is not a value in the array
Plotted -0.31 instead


In [542]:
plt.figure()
plt.pcolormesh(barr-b02, nvals2928_600, dG600_865, vmin=-1, vmax=0.1)
plt.colorbar()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [543]:
plt.figure()
plt.plot(barr-b02, np.average(dG600_865[:], axis=0))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Holes 3K

In [544]:
zeroDholef3k = sorted(glob.glob('data/2020-06-13/#*/'))[85:] + sorted(glob.glob('data/2020-06-14/#*/'))[1:2]

In [545]:
Vbvals = np.linspace(-3.5, -3.1, 41)
nvals2928_3k = nzero2928(Vbvals)
nvals4041_3k = nzero4041(Vbvals)

In [558]:
# raw Resistance data placed into an array
walarr3k_865 = np.zeros((len(nvals2928_3k), len(barr)))
for i, f in enumerate(zeroDholef3k):
    d = qc.load_data(f[:-1])
    dlast = np.where(np.isnan(d.triton_field))[0][0]
    sweep = np.interp(barr/1000, d.triton_field[:dlast], d.lockin865_X[:dlast]/10e-9)
    walarr3k_865[i, :] = sweep

# form an array of change in conductance from the zero point
b02 = -0.31  # the offset of the field zero point
pind = npd.val_to_index([b02], barr)[0]
dG3k_865 = np.zeros(walarr3k_865.shape)
for i in range(walarr3k_865.shape[0]):
    dG3k_865[i, :] = 1/walarr3k_865[i, :]/esqh - 1/walarr3k_865[i, pind]/esqh

-0.31 is not a value in the array
Plotted -0.31 instead


In [559]:
plt.figure()
p = plt.plot(barr-b02, (walarr3k_865 - np.transpose([np.average(walarr3k_865, axis=1),]*walarr3k_865.shape[1])).T, lw=0.5)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [560]:
plt.figure()
plt.pcolormesh(barr-b02, nvals2928_3k, walarr3k_865 - np.transpose([np.average(walarr3k_865, axis=1),]*walarr3k_865.shape[1]))
plt.colorbar()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [561]:
plt.figure()
plt.pcolormesh(barr-b02, nvals2928_3k, dG3k_865, vmin=-1, vmax=0.1)
plt.colorbar()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [563]:
plt.figure()
plt.plot(barr-b02, np.average(dG3k_865[:], axis=0))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Holes combined

In [567]:
plt.figure()
plt.plot(barr-b02, np.average(dG2_865[10:], axis=0), label='Base T')
plt.plot(barr-b02, np.average(dG300_865[10:], axis=0), label='300 mK')
plt.plot(barr-b02, np.average(dG600_865[10:], axis=0), label='600 mK')
plt.plot(barr-b02, np.average(dG3k_865[10:], axis=0), label='3 K')
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [570]:
plt.figure()
plt.plot(barr-b02, np.average(dG2_865[10:], axis=0) - np.average(dG3k_865[20:], axis=0), label='Base T')
plt.plot(barr-b02, np.average(dG300_865[10:], axis=0) - np.average(dG3k_865[20:], axis=0), label='300 mK')
plt.plot(barr-b02, np.average(dG600_865[10:], axis=0) - np.average(dG3k_865[20:], axis=0), label='600 mK')
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Fine scan electrons

In [343]:
zeroDelf = sorted(glob.glob('data/2020-06-07/#*/'))[120:] + sorted(glob.glob('data/2020-06-08/#*/'))[:21]
L = len(zeroDelf)
L

51

In [344]:
Vbvals = np.linspace(-2.9, -2.4, L)
nvals2928_3 = nzero2928(Vbvals)
nvals4041_3 = nzero4041(Vbvals)

In [345]:
# raw Resistance data placed into an array
walarr3_865 = np.zeros((L, len(barr)))
for i, f in enumerate(zeroDelf):
    d = qc.load_data(f[:-1])
    dlast = np.where(np.isnan(d.triton_field))[0][0]
    sweep = np.interp(barr/1000, d.triton_field[:dlast], d.lockin865_X[:dlast]/10e-9)
    walarr3_865[i, :] = sweep

# form an array of change in conductance from the zero point
b03 = -0.58  # the offset of the field zero point
pind = npd.val_to_index([b0], barr)[0]
dG3_865 = np.zeros(walarr3_865.shape)
for i in range(L):
    dG3_865[i, :] = 1/walarr3_865[i, :]/esqh - 1/walarr3_865[i, pind] /esqh

-0.58 is not a value in the array
Plotted -0.62 instead


In [346]:
plt.figure()
plt.pcolormesh(barr-b03, nvals2928_3, dG3_865, vmin=-1, vmax=0.1)
plt.colorbar()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [347]:
plt.figure()
p = plt.plot(barr-b03, dG3_865.T, lw=0.2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [348]:
plt.figure()
plt.plot(barr-b03, np.average(dG3_865[:], axis=0), label='all')
plt.plot(barr-b03, np.average(dG3_865[:20], axis=0), label='sweeps 0 to 20')
plt.plot(barr-b03, np.average(dG3_865[20:40], axis=0), label='sweeps 20 to 40')
plt.plot(barr-b03, np.average(dG3_865[30:], axis=0), label='sweeps 30 to 50')
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Red line  (-0.70 V/nm)

### Hole side base T

In [479]:
def nred2928(Vb):
    # the density (in 10^12 cm^-2) for electrodes 29-28, on the red line, given Vb
    return nD2928(Vb, -slDrc*Vb - 2.2)[0]

def nred4041(Vb):
    # the density (in 10^12 cm^-2) for electrodes 29-28, on the the red line, given Vb
    return nD4041(Vb, -slDrc*Vb - 2.2)[0]

In [480]:
redDholef = sorted(glob.glob('data/2020-06-08/#*/'))[21:62]

In [481]:
Vbvals = np.linspace(1.84, 2.24, 41)
nvals2928_hr = nred2928(Vbvals)
nvals4041_hr = nred4041(Vbvals)

In [482]:
# raw Resistance data placed into an array
walarrhr_865 = np.zeros((len(nvals2928_hr), len(barr)))
for i, f in enumerate(redDholef):
    d = qc.load_data(f[:-1])
    dlast = np.where(np.isnan(d.triton_field))[0][0]
    sweep = np.interp(barr/1000, d.triton_field[:dlast], d.lockin865_X[:dlast]/10e-9)
    walarrhr_865[i, :] = sweep

# form an array of change in conductance from the zero point
b02 = -0.31  # the offset of the field zero point
pind = npd.val_to_index([b0], barr)[0]
dGhr_865 = np.zeros(walarrhr_865.shape)
for i in range(walarrhr_865.shape[0]):
    dGhr_865[i, :] = 1/walarrhr_865[i, :]/esqh - 1/walarrhr_865[i, pind]/esqh

-0.58 is not a value in the array
Plotted -0.62 instead


In [483]:
plt.figure()
plt.pcolormesh(barr-b02, nvals2928_hr, dGhr_865, vmin=-1, vmax=0.1)
plt.colorbar()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [484]:
plt.figure()
plt.plot(barr-b02, np.average(dGhr_865[:], axis=0), label='all')
plt.plot(barr-b02, np.average(dGhr_865[:20], axis=0), label='sweeps 0 to 20')
plt.plot(barr-b02, np.average(dGhr_865[20:40], axis=0), label='sweeps 20 to 40')
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Holes 300 mK

In [454]:
redDholef300 = sorted(glob.glob('data/2020-06-11/#*/'))[85:106]
redDholef300

['data/2020-06-11/#086_M30_865_28-29_830_40-41_300mK_WALfieldsw_redline_Vbg1.84_10nA_19-16-16/',
 'data/2020-06-11/#087_M30_865_28-29_830_40-41_300mK_WALfieldsw_redline_Vbg1.86_10nA_19-27-21/',
 'data/2020-06-11/#088_M30_865_28-29_830_40-41_300mK_WALfieldsw_redline_Vbg1.88_10nA_19-38-27/',
 'data/2020-06-11/#089_M30_865_28-29_830_40-41_300mK_WALfieldsw_redline_Vbg1.90_10nA_19-49-35/',
 'data/2020-06-11/#090_M30_865_28-29_830_40-41_300mK_WALfieldsw_redline_Vbg1.92_10nA_20-00-39/',
 'data/2020-06-11/#091_M30_865_28-29_830_40-41_300mK_WALfieldsw_redline_Vbg1.94_10nA_20-11-45/',
 'data/2020-06-11/#092_M30_865_28-29_830_40-41_300mK_WALfieldsw_redline_Vbg1.96_10nA_20-22-51/',
 'data/2020-06-11/#093_M30_865_28-29_830_40-41_300mK_WALfieldsw_redline_Vbg1.98_10nA_20-33-56/',
 'data/2020-06-11/#094_M30_865_28-29_830_40-41_300mK_WALfieldsw_redline_Vbg2.00_10nA_20-45-01/',
 'data/2020-06-11/#095_M30_865_28-29_830_40-41_300mK_WALfieldsw_redline_Vbg2.02_10nA_20-56-08/',
 'data/2020-06-11/#096_M30_865

In [455]:
Vbvals = np.linspace(1.84, 2.24, 21)
nvals2928_hr300 = nred2928(Vbvals)
nvals4041_hr300 = nred4041(Vbvals)

In [456]:
# raw Resistance data placed into an array
walarrhr300_865 = np.zeros((len(nvals2928_hr300), len(barr)))
for i, f in enumerate(redDholef300):
    d = qc.load_data(f[:-1])
    dlast = np.where(np.isnan(d.triton_field))[0][0]
    sweep = np.interp(barr/1000, d.triton_field[:dlast], d.lockin865_X[:dlast]/10e-9)
    walarrhr300_865[i, :] = sweep

# form an array of change in conductance from the zero point
b02 = -0.31  # the offset of the field zero point
pind = npd.val_to_index([b0], barr)[0]
dGhr300_865 = np.zeros(walarrhr300_865.shape)
for i in range(walarrhr300_865.shape[0]):
    dGhr300_865[i, :] = 1/walarrhr300_865[i, :]/esqh - 1/walarrhr300_865[i, pind]/esqh

-0.58 is not a value in the array
Plotted -0.62 instead


In [457]:
plt.figure()
plt.pcolormesh(barr-b02, nvals2928_hr300, dGhr300_865, vmin=-1, vmax=0.1)
plt.colorbar()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [458]:
plt.figure()
plt.plot(barr-b02, np.average(dGhr300_865[:], axis=0))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Holes 600 mK

In [462]:
redDholef600 = sorted(glob.glob('data/2020-06-12/#*/'))[115:] + sorted(glob.glob('data/2020-06-13/#*/'))[:12]

In [463]:
Vbvals = np.linspace(1.84, 2.24, 21)
nvals2928_hr600 = nred2928(Vbvals)
nvals4041_hr600 = nred4041(Vbvals)

In [467]:
# raw Resistance data placed into an array
walarrhr600_865 = np.zeros((len(nvals2928_hr600), len(barr)))
for i, f in enumerate(redDholef600):
    d = qc.load_data(f[:-1])
    dlast = np.where(np.isnan(d.triton_field))[0][0]
    sweep = np.interp(barr/1000, d.triton_field[:dlast], d.lockin865_X[:dlast]/10e-9)
    walarrhr600_865[i, :] = sweep

# form an array of change in conductance from the zero point
b02 = -0.31  # the offset of the field zero point
pind = npd.val_to_index([b0], barr)[0]
dGhr600_865 = np.zeros(walarrhr600_865.shape)
for i in range(walarrhr600_865.shape[0]):
    dGhr600_865[i, :] = 1/walarrhr600_865[i, :]/esqh - 1/walarrhr600_865[i, pind]/esqh

-0.58 is not a value in the array
Plotted -0.62 instead


In [468]:
plt.figure()
plt.pcolormesh(barr-b02, nvals2928_hr600, dGhr600_865, vmin=-1, vmax=0.1)
plt.colorbar()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [469]:
plt.figure()
plt.plot(barr-b02, np.average(dGhr600_865[:], axis=0))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Holes 3K

In [472]:
redDholef3k = sorted(glob.glob('data/2020-06-14/#*/'))[53:94]

In [474]:
Vbvals = np.linspace(1.84, 2.24, 41)
nvals2928_hr3k = nred2928(Vbvals)
nvals4041_hr3k = nred4041(Vbvals)

In [475]:
# raw Resistance data placed into an array
walarrhr3k_865 = np.zeros((len(nvals2928_hr3k), len(barr)))
for i, f in enumerate(redDholef3k):
    d = qc.load_data(f[:-1])
    dlast = np.where(np.isnan(d.triton_field))[0][0]
    sweep = np.interp(barr/1000, d.triton_field[:dlast], d.lockin865_X[:dlast]/10e-9)
    walarrhr3k_865[i, :] = sweep

# form an array of change in conductance from the zero point
b02 = -0.31  # the offset of the field zero point
pind = npd.val_to_index([b0], barr)[0]
dGhr3k_865 = np.zeros(walarrhr3k_865.shape)
for i in range(walarrhr3k_865.shape[0]):
    dGhr3k_865[i, :] = 1/walarrhr3k_865[i, :]/esqh - 1/walarrhr3k_865[i, pind]/esqh

-0.58 is not a value in the array
Plotted -0.62 instead


In [476]:
plt.figure()
plt.pcolormesh(barr-b02, nvals2928_hr3k, dGhr3k_865, vmin=-1, vmax=0.1)
plt.colorbar()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [477]:
plt.figure()
plt.plot(barr-b02, np.average(dGhr3k_865[:], axis=0))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Holes Temps combined 

In [485]:
plt.figure()
plt.plot(barr-b02, np.average(dGhr_865[:], axis=0), label='Base T')
plt.plot(barr-b02, np.average(dGhr300_865[:], axis=0), label='300 mK')
plt.plot(barr-b02, np.average(dGhr600_865[:], axis=0), label='600 mK')
plt.plot(barr-b02, np.average(dGhr3k_865[:], axis=0), label='3 K')
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [489]:
# subtract the 3K data
plt.figure()
plt.plot(barr-b02, np.average(dGhr_865[:], axis=0) - np.average(dGhr3k_865[:], axis=0), label='Base T')
plt.plot(barr-b02, np.average(dGhr300_865[:], axis=0) - np.average(dGhr3k_865[:], axis=0), label='300 mK')
plt.plot(barr-b02, np.average(dGhr600_865[:], axis=0) - np.average(dGhr3k_865[:], axis=0), label='600 mK')
# plt.plot(barr-b02, np.average(dGhr3k_865[:], axis=0), label='3 K')
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Electron side

In [375]:
redDelf = sorted(glob.glob('data/2020-06-08/#*/'))[62:113]

In [371]:
Vbvals = np.linspace(2.44, 2.94, 51)
nvals2928_er = nred2928(Vbvals)
nvals4041_er = nred4041(Vbvals)

In [372]:
# raw Resistance data placed into an array
walarrer_865 = np.zeros((len(nvals2928_er), len(barr)))
for i, f in enumerate(redDelf):
    d = qc.load_data(f[:-1])
    dlast = np.where(np.isnan(d.triton_field))[0][0]
    sweep = np.interp(barr/1000, d.triton_field[:dlast], d.lockin865_X[:dlast]/10e-9)
    walarrer_865[i, :] = sweep

# form an array of change in conductance from the zero point
b02 = -0.31  # the offset of the field zero point
pind = npd.val_to_index([b0], barr)[0]
dGer_865 = np.zeros(walarrer_865.shape)
for i in range(walarrer_865.shape[0]):
    dGer_865[i, :] = 1/walarrer_865[i, :]/esqh - 1/walarrer_865[i, pind]/esqh

-0.58 is not a value in the array
Plotted -0.62 instead


In [373]:
plt.figure()
plt.pcolormesh(barr-b02, nvals2928_er, dGer_865, vmin=-1, vmax=0.1)
plt.colorbar()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [374]:
plt.figure()
plt.plot(barr-b02, np.average(dGer_865[:], axis=0), label='all')
plt.plot(barr-b02, np.average(dGer_865[:20], axis=0), label='sweeps 0 to 20')
plt.plot(barr-b02, np.average(dGer_865[20:40], axis=0), label='sweeps 20 to 40')
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Orange line (0.925 V/nm)

In [376]:
def norange2928(Vb):
    # the density (in 10^12 cm^-2) for electrodes 29-28, on the orange line, given Vb
    return nD2928(Vb, -slDrc*Vb + 2.9)[0]

def norange4041(Vb):
    # the density (in 10^12 cm^-2) for electrodes 29-28, on the the orange line, given Vb
    return nD4041(Vb, -slDrc*Vb + 2.9)[0]

### Hole side base T

In [580]:
orangeDholef = sorted(glob.glob('data/2020-06-08/#*/'))[113:] + sorted(glob.glob('data/2020-06-09/#*/'))[:25]

In [581]:
Vbvals = np.linspace(-10.55, -10.15, 41)
nvals2928_ho = norange2928(Vbvals)
nvals4041_ho = norange4041(Vbvals)

In [582]:
# raw Resistance data placed into an array
walarrho_865 = np.zeros((len(nvals2928_ho), len(barr)))
for i, f in enumerate(orangeDholef):
    d = qc.load_data(f[:-1])
    dlast = np.where(np.isnan(d.triton_field))[0][0]
    sweep = np.interp(barr/1000, d.triton_field[:dlast], d.lockin865_X[:dlast]/10e-9)
    walarrho_865[i, :] = sweep

# form an array of change in conductance from the zero point
b02 = -0.31  # the offset of the field zero point
pind = npd.val_to_index([b02], barr)[0]
dGho_865 = np.zeros(walarrho_865.shape)
for i in range(walarrho_865.shape[0]):
    dGho_865[i, :] = 1/walarrho_865[i, :]/esqh - 1/walarrho_865[i, pind]/esqh

-0.31 is not a value in the array
Plotted -0.31 instead


In [583]:
plt.figure()
plt.pcolormesh(barr-b02, nvals2928_ho, dGho_865, vmin=-1, vmax=0.1)
plt.colorbar()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [584]:
plt.figure()
plt.plot(barr-b02, np.average(dGho_865[:], axis=0), label='all')
plt.plot(barr-b02, np.average(dGho_865[:20], axis=0), label='sweeps 0 to 20')
plt.plot(barr-b02, np.average(dGho_865[20:40], axis=0), label='sweeps 20 to 40')
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Holes side 300 mK

In [585]:
orangeDholef300 = sorted(glob.glob('data/2020-06-12/#*/'))[21:42]

In [587]:
Vbvals = np.linspace(-10.55, -10.15, 21)
nvals2928_ho300 = norange2928(Vbvals)
nvals4041_ho300 = norange4041(Vbvals)

In [588]:
# raw Resistance data placed into an array
walarrho300_865 = np.zeros((len(nvals2928_ho300), len(barr)))
for i, f in enumerate(orangeDholef300):
    d = qc.load_data(f[:-1])
    dlast = np.where(np.isnan(d.triton_field))[0][0]
    sweep = np.interp(barr/1000, d.triton_field[:dlast], d.lockin865_X[:dlast]/10e-9)
    walarrho300_865[i, :] = sweep

# form an array of change in conductance from the zero point
b02 = -0.31  # the offset of the field zero point
pind = npd.val_to_index([b02], barr)[0]
dGho300_865 = np.zeros(walarrho300_865.shape)
for i in range(walarrho300_865.shape[0]):
    dGho300_865[i, :] = 1/walarrho300_865[i, :]/esqh - 1/walarrho300_865[i, pind]/esqh

-0.31 is not a value in the array
Plotted -0.31 instead


In [589]:
plt.figure()
plt.pcolormesh(barr-b02, nvals2928_ho300, dGho300_865, vmin=-1, vmax=0.1)
plt.colorbar()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [590]:
plt.figure()
plt.plot(barr-b02, np.average(dGho300_865[:], axis=0), label='all')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Hole side 600 mK

In [594]:
orangeDholef600 = sorted(glob.glob('data/2020-06-13/#*/'))[38:59]

In [595]:
Vbvals = np.linspace(-10.55, -10.15, 21)
nvals2928_ho600 = norange2928(Vbvals)
nvals4041_ho600 = norange4041(Vbvals)

In [596]:
# raw Resistance data placed into an array
walarrho600_865 = np.zeros((len(nvals2928_ho600), len(barr)))
for i, f in enumerate(orangeDholef600):
    d = qc.load_data(f[:-1])
    dlast = np.where(np.isnan(d.triton_field))[0][0]
    sweep = np.interp(barr/1000, d.triton_field[:dlast], d.lockin865_X[:dlast]/10e-9)
    walarrho600_865[i, :] = sweep

# form an array of change in conductance from the zero point
b02 = -0.31  # the offset of the field zero point
pind = npd.val_to_index([b02], barr)[0]
dGho600_865 = np.zeros(walarrho600_865.shape)
for i in range(walarrho600_865.shape[0]):
    dGho600_865[i, :] = 1/walarrho600_865[i, :]/esqh - 1/walarrho600_865[i, pind]/esqh

-0.31 is not a value in the array
Plotted -0.31 instead


In [597]:
plt.figure()
plt.pcolormesh(barr-b02, nvals2928_ho600, dGho600_865, vmin=-1, vmax=0.1)
plt.colorbar()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [598]:
plt.figure()
plt.plot(barr-b02, np.average(dGho600_865[:], axis=0), label='all')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Holes 3K

In [600]:
orangeDholef3k = sorted(glob.glob('data/2020-06-15/#*/'))[20:61]

In [601]:
Vbvals = np.linspace(-10.55, -10.15, 41)
nvals2928_ho3k = norange2928(Vbvals)
nvals4041_ho3k = norange4041(Vbvals)

In [602]:
# raw Resistance data placed into an array
walarrho3k_865 = np.zeros((len(nvals2928_ho3k), len(barr)))
for i, f in enumerate(orangeDholef3k):
    d = qc.load_data(f[:-1])
    dlast = np.where(np.isnan(d.triton_field))[0][0]
    sweep = np.interp(barr/1000, d.triton_field[:dlast], d.lockin865_X[:dlast]/10e-9)
    walarrho3k_865[i, :] = sweep

# form an array of change in conductance from the zero point
b02 = -0.31  # the offset of the field zero point
pind = npd.val_to_index([b02], barr)[0]
dGho3k_865 = np.zeros(walarrho3k_865.shape)
for i in range(walarrho3k_865.shape[0]):
    dGho3k_865[i, :] = 1/walarrho3k_865[i, :]/esqh - 1/walarrho3k_865[i, pind]/esqh

-0.31 is not a value in the array
Plotted -0.31 instead


In [603]:
plt.figure()
plt.pcolormesh(barr-b02, nvals2928_ho3k, dGho3k_865, vmin=-1, vmax=0.1)
plt.colorbar()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [604]:
plt.figure()
plt.plot(barr-b02, np.average(dGho3k_865[:], axis=0), label='all')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Combined

In [605]:
plt.figure()
plt.plot(barr-b02, np.average(dGho_865[:], axis=0), label='Base T')
plt.plot(barr-b02, np.average(dGho300_865[:], axis=0), label='300 mK')
plt.plot(barr-b02, np.average(dGho600_865[:], axis=0), label='600 mK')
plt.plot(barr-b02, np.average(dGho3k_865[:], axis=0), label='3 K')
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [607]:
# subtract 3K
plt.figure()
plt.plot(barr-b02, np.average(dGho_865[:], axis=0) - np.average(dGho3k_865[:], axis=0), label='Base T')
plt.plot(barr-b02, np.average(dGho300_865[:], axis=0) - np.average(dGho3k_865[:], axis=0), label='300 mK')
plt.plot(barr-b02, np.average(dGho600_865[:], axis=0) - np.average(dGho3k_865[:], axis=0), label='600 mK')
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Electron side

In [404]:
orangeDelf = sorted(glob.glob('data/2020-06-09/#*/'))[25:60] + sorted(glob.glob('data/2020-06-09/#*/'))[68:84]

In [405]:
Vbvals = np.linspace(-9.95, -9.45, 51)
nvals2928_eo = norange2928(Vbvals)
nvals4041_eo = norange4041(Vbvals)

In [410]:
# raw Resistance data placed into an array
walarreo_865 = np.zeros((len(nvals2928_eo), len(barr)))
for i, f in enumerate(orangeDelf):
    d = qc.load_data(f[:-1])
    dlast = np.where(np.isnan(d.triton_field))[0][0]
    sweep = np.interp(barr/1000, d.triton_field[:dlast], d.lockin865_X[:dlast]/10e-9)
    walarreo_865[i, :] = sweep

# form an array of change in conductance from the zero point
b02 = -0.31  # the offset of the field zero point
pind = npd.val_to_index([b0], barr)[0]
dGeo_865 = np.zeros(walarreo_865.shape)
for i in range(walarreo_865.shape[0]):
    dGeo_865[i, :] = 1/walarreo_865[i, :]/esqh - 1/walarreo_865[i, pind]/esqh

-0.58 is not a value in the array
Plotted -0.62 instead


In [411]:
plt.figure()
plt.pcolormesh(barr-b02, nvals2928_eo, dGeo_865, vmin=-1, vmax=0.1)
plt.colorbar()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [414]:
plt.figure()
plt.plot(barr-b02, np.average(dGeo_865[:], axis=0), label='all')
plt.plot(barr-b02, np.average(dGeo_865[:20], axis=0), label='sweeps 0 to 20')
plt.plot(barr-b02, np.average(dGeo_865[20:40], axis=0), label='sweeps 20 to 40')
plt.plot(barr-b02, np.average(dGeo_865[30:50], axis=0), label='sweeps 30 to 50')
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Rxy and Hall density plots

In [48]:
d0rxyfan = qc.load_data('data/2020-05-15/#005_M30_865_28-42__830_32-40_1nA_baseT_B-n_D0_sweep_21-41-18')
d0rxyfan

DataSet:
   location = 'data/2020-05-15/#005_M30_865_28-42__830_32-40_1nA_baseT_B-n_D0_sweep_21-41-18'
   <Type>   | <array_id>                  | <array.name>     | <array.shape>
   Setpoint | triton_field_set_stable_set | field_set_stable | (161,)
   Setpoint | zeroDparam_set              | zeroDparam       | (161, 451)
   Measured | lockin830_X                 | X                | (161, 451)
   Measured | lockin830_Y                 | Y                | (161, 451)
   Measured | lockin865_X                 | X                | (161, 451)
   Measured | lockin865_Y                 | Y                | (161, 451)

In [49]:
d0rxxfan = qc.load_data('data/2020-05-06/#005_M30_865_29-28__830_40-41_1nA_baseT_B-n_D0_sweep_23-13-35')
Rxx0 = npd.Rxxfromdata(d0rxxfan, 1e-9, 'lockin865')
d0rxxfan.triton_field_set_stable_set

DataArray[161]: triton_field_set_stable_set
array([0.  , 0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 ,
       0.55, 0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95, 1.  , 1.05,
       1.1 , 1.15, 1.2 , 1.25, 1.3 , 1.35, 1.4 , 1.45, 1.5 , 1.55, 1.6 ,
       1.65, 1.7 , 1.75, 1.8 , 1.85, 1.9 , 1.95, 2.  , 2.05, 2.1 , 2.15,
       2.2 , 2.25, 2.3 , 2.35, 2.4 , 2.45, 2.5 , 2.55, 2.6 , 2.65, 2.7 ,
       2.75, 2.8 , 2.85, 2.9 , 2.95, 3.  , 3.05, 3.1 , 3.15, 3.2 , 3.25,
       3.3 , 3.35, 3.4 , 3.45, 3.5 , 3.55, 3.6 , 3.65, 3.7 , 3.75, 3.8 ,
       3.85, 3.9 , 3.95, 4.  , 4.05, 4.1 , 4.15, 4.2 , 4.25, 4.3 , 4.35,
       4.4 , 4.45, 4.5 , 4.55, 4.6 , 4.65, 4.7 , 4.75, 4.8 , 4.85, 4.9 ,
       4.95, 5.  , 5.05, 5.1 , 5.15, 5.2 , 5.25, 5.3 , 5.35, 5.4 , 5.45,
       5.5 , 5.55, 5.6 , 5.65, 5.7 , 5.75, 5.8 , 5.85, 5.9 , 5.95, 6.  ,
       6.05, 6.1 , 6.15, 6.2 , 6.25, 6.3 , 6.35, 6.4 , 6.45, 6.5 , 6.55,
       6.6 , 6.65, 6.7 , 6.75, 6.8 , 6.85, 6.9 , 6.95, 7.  , 7.05, 7.1 ,
       

In [50]:
d0rxxfan

DataSet:
   location = 'data/2020-05-06/#005_M30_865_29-28__830_40-41_1nA_baseT_B-n_D0_sweep_23-13-35'
   <Type>   | <array_id>                  | <array.name>     | <array.shape>
   Setpoint | triton_field_set_stable_set | field_set_stable | (161,)
   Setpoint | zeroDparam_set              | zeroDparam       | (161, 451)
   Measured | lockin830_X                 | X                | (161, 451)
   Measured | lockin830_Y                 | Y                | (161, 451)
   Measured | lockin865_X                 | X                | (161, 451)
   Measured | lockin865_Y                 | Y                | (161, 451)

In [51]:
d0rxyfan.lockin830_X[:, :] *= -1

In [52]:
esqh = 1.602e-19**2/6.626e-34

In [53]:
d0rxyfan.triton_field_set_stable_set

DataArray[161]: triton_field_set_stable_set
array([0.  , 0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 ,
       0.55, 0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95, 1.  , 1.05,
       1.1 , 1.15, 1.2 , 1.25, 1.3 , 1.35, 1.4 , 1.45, 1.5 , 1.55, 1.6 ,
       1.65, 1.7 , 1.75, 1.8 , 1.85, 1.9 , 1.95, 2.  , 2.05, 2.1 , 2.15,
       2.2 , 2.25, 2.3 , 2.35, 2.4 , 2.45, 2.5 , 2.55, 2.6 , 2.65, 2.7 ,
       2.75, 2.8 , 2.85, 2.9 , 2.95, 3.  , 3.05, 3.1 , 3.15, 3.2 , 3.25,
       3.3 , 3.35, 3.4 , 3.45, 3.5 , 3.55, 3.6 , 3.65, 3.7 , 3.75, 3.8 ,
       3.85, 3.9 , 3.95, 4.  , 4.05, 4.1 , 4.15, 4.2 , 4.25, 4.3 , 4.35,
       4.4 , 4.45, 4.5 , 4.55, 4.6 , 4.65, 4.7 , 4.75, 4.8 , 4.85, 4.9 ,
       4.95, 5.  , 5.05, 5.1 , 5.15, 5.2 , 5.25, 5.3 , 5.35, 5.4 , 5.45,
       5.5 , 5.55, 5.6 , 5.65, 5.7 , 5.75, 5.8 , 5.85, 5.9 , 5.95, 6.  ,
       6.05, 6.1 , 6.15, 6.2 , 6.25, 6.3 , 6.35, 6.4 , 6.45, 6.5 , 6.55,
       6.6 , 6.65, 6.7 , 6.75, 6.8 , 6.85, 6.9 , 6.95, 7.  , 7.05, 7.1 ,
       

In [54]:
fig, ax = plt.subplots(2, 1, figsize=(9, 7), sharex=True)
ind = 10
field = d0rxyfan.triton_field_set_stable_set[ind]
changerxy = gaussian_filter((d0rxyfan.lockin865_X[ind, :] - d0rxyfan.lockin865_X[0, :])/1e-9, 1.2)
ax[0].plot(d0rxyfan.zeroDparam_set[0, :], field/(1.602e-19*changerxy)/10**16, label='B = {}T'.format(field))
ax[0].set_ylim(-2, 2)
ax[0].legend()
ax[1].semilogy(d0rxxfan.zeroDparam_set[0, :], Rxx0[0, :])
ax[1].set_ylim(50, 10**5)
ax[1].set_xlabel('V$_{bg}$ (topgate also varied for constant D = 0)')
ax[0].set_ylabel('n$_{Hall}$ (10$^{12}$ cm$^{-2}$)')
ax[1].set_ylabel('R$_{xx}$ ($\Omega$)')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'R$_{xx}$ ($\\Omega$)')

Try fitting a line to the points from 0T to 0.5T for each gate point

In [55]:
gate = d0rxyfan.zeroDparam_set[0, :]
drxydb = np.zeros(len(gate))
for i in range(len(gate)):
    sl, b = np.polyfit([0, 0.1, 0.2, 0.3, 0.4, 0.5], d0rxyfan.lockin865_X[:6, i]/1e-9, 1)
    drxydb[i] = sl

In [56]:
fig, ax = plt.subplots(2, 1, figsize=(9, 7), sharex=True)
ind = 10
nHall = 1/(1.602e-19*drxydb)/10**16
ax[0].plot(gate, gaussian_filter(nHall, 1))
ax[0].set_ylim(-5, 5)
ax[1].semilogy(gate, Rxx0[0, :])
ax[1].set_ylim(50, 10**5)
ax[1].set_xlabel('V$_{bg}$ (topgate also varied for constant D = 0)')
ax[0].set_ylabel('n$_{Hall}$ (10$^{12}$ cm$^{-2}$)')
ax[1].set_ylabel('R$_{xx}$ ($\Omega$)')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'R$_{xx}$ ($\\Omega$)')

Try taking out 0T (maybe also 0.1T) and do some filtering in the gate-Rxy curves before fitting the slopes

In [57]:
gate = d0rxyfan.zeroDparam_set[0, :]
drxyfanfilt = gaussian_filter(d0rxyfan.lockin865_X[2:7, :]/1e-9, (0, 1))

drxydb = np.zeros(len(gate))
for i in range(len(gate)):
    sl, b = np.polyfit([0.2, 0.3, 0.4, 0.5, 0.6], drxyfanfilt[:, i], 1)
    drxydb[i] = sl

In [58]:
fig, ax = plt.subplots(2, 1, figsize=(9, 7), sharex=True)
ind = 10
nHall = 1/(1.602e-19*drxydb)/10**16
ax[0].plot(gate, gaussian_filter(nHall, 1))
ax[0].set_ylim(-7, 7)
ax[1].semilogy(gate, Rxx0[0, :])
ax[1].set_ylim(50, 10**5)
ax[1].set_xlabel('V$_{bg}$ (topgate also varied for constant D = 0)')
ax[0].set_ylabel('n$_{Hall}$ (10$^{12}$ cm$^{-2}$)')
ax[1].set_ylabel('R$_{xx}$ ($\Omega$)')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'R$_{xx}$ ($\\Omega$)')